In [25]:
import pandas as pd
import numpy as np
import os 
from datetime import datetime
import scipy as sp
import time
from datetime import datetime, date, timedelta

from dateutil.relativedelta import relativedelta

import io
from io import StringIO 
#import datedelta
import calendar
import  csv
import json
import random

import plotly
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.io as pio
from scipy import stats
def add_delta_cols(df):
    #df['delta_vector'] = 0
    #df['delta_max_vector'] = 0

    df['delta_vector'] = 0
    df['delta_max'] = 0
    df['upper_wick'] = 0
    df['lower_wick'] = 0
    
    dv_col = df.columns.get_loc('delta_vector')
    dm_col = df.columns.get_loc('delta_max')
    uw_col = df.columns.get_loc('upper_wick')
    lw_col = df.columns.get_loc('lower_wick')
    
    o_col = df.columns.get_loc('o')
    h_col = df.columns.get_loc('h')
    l_col = df.columns.get_loc('l')
    c_col = df.columns.get_loc('c')
    arr = df.values
    for i in range(arr.shape[0]):
        arr[i,dv_col] = arr[i,c_col] - arr[i,o_col]
        arr[i,dm_col] = arr[i,h_col] - arr[i,l_col]
        if arr[i,c_col] > arr[i,o_col]:
            arr[i,uw_col] = arr[i,h_col] - arr[i,c_col]
            arr[i,lw_col] = arr[i,o_col] - arr[i,l_col]
            
        elif arr[i,c_col] < arr[i,o_col]:
            arr[i,uw_col] = arr[i,h_col] - arr[i,o_col]
            arr[i,lw_col] = arr[i,c_col] - arr[i,l_col]   
            
    df = pd.DataFrame(arr,columns = df.columns)
    df['delta'] = abs(df['delta_vector'])
    
    return df
def load_df(pair = 'EUR_USD'
            ,granularity = 'M5'
            ,start = datetime(2016,1,1,0,0,0)
            ,end = datetime(2022,7,31,0,0,0)):
    dir_name = os.getcwd() + '/' + pair + '_' + granularity
    path = dir_name + '/' + str(date(start.year,start.month,start.day)) + '_' + str(date(end.year,end.month,end.day)) + '.csv'
    print('PATH:',path)
    df = pd.read_csv(path)
    df = add_delta_cols(df)
    print(df.shape)
    return df
def get_support(df,
                lookup_range = 1500,
                lookup_range2 = 2400,
                stop_range = 2200,
                candle_size = .001,
                bound_pips = .001
               ):
    print('GETTING SUPPORT INDICATOR')
    s = time.time()
    c_col = df.columns.get_loc('c')
    o_col = df.columns.get_loc('o')
    l_col = df.columns.get_loc('l')
    h_col = df.columns.get_loc('h')
    d_col = df.columns.get_loc('delta')
    df['support_lookup'] = 0
    df['support_indicator'] = 0

    lookup_col = df.columns.get_loc('support_lookup')
    new_col = df.columns.get_loc('support_indicator')

    reversal_col = df.columns.get_loc('c')   

    arr = df.values
    min_count = 0

    print('STOP RANGE:',stop_range)
    c = 0 

    for i in range(lookup_range,arr.shape[0]):
       # try:
        #DEFINE SUPPORT/RESISTANCE as the MIN or MAX of a lookup range
        min_ = min(arr[i - lookup_range : i + lookup_range2,reversal_col])

        # If the current val is at that reversal point:
        # Check and make sure we have an adequate movement off of that reversal point
        # this is defined by the pip_range and pip threshold, pip threshold is a user defined parameter
        if arr[i,reversal_col] == min_ and (i + lookup_range2 + stop_range < arr.shape[0]):
            lookup_ind = i
            val = arr[i,reversal_col]
            lookup_check = 0
            c = c + 1
            # Now that all of these conditions are met
            # We need to look into the future to see if we hit this reversal level, this is defined by the stop range
            for j in range(i + lookup_range2,i + lookup_range2 + stop_range):
                #If all of the following conditions are met, we will mark the array with a 1 signal to indicated we would place a trade                       
                #CANDLE BULL OR BEAR
                if arr[j,c_col] >= arr[j,o_col]:
                    bull = True
                else:
                    bull = False                        
                lower_bound = val - bound_pips
                upper_bound = val + bound_pips
                if i + lookup_range2 == j:
                    min_val = arr[j, reversal_col]
                else:
                    min_val = min(arr[i + lookup_range2 : j, reversal_col])
                    
                cur_val = arr[j - 1, reversal_col]

                if lookup_check == 0 and \
                bull == True and\
                arr[j,d_col] >= candle_size and\
                min_val >= lower_bound and \
                cur_val <= upper_bound and cur_val >= lower_bound:
                    arr[j,new_col] = 1
                    arr[j,lookup_col] = lookup_ind
                    lookup_check = 1
                    break

       # except:
        #    pass

    print('COUNT:',c)

    e = time.time()
    print('TOTAL FUNCTION TIME:',(e-s)/60,' MINUTES')
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['support_indicator'] == 1].shape)
    return df

def get_resistance(df,
                lookup_range = 1500,
                lookup_range2 = 2400,
                stop_range = 2200,
                candle_size = .001,
                bound_pips = .001
               ):
    print('GETTING RESISTANCE INDICATOR')
    s = time.time()
    c_col = df.columns.get_loc('c')
    o_col = df.columns.get_loc('o')
    l_col = df.columns.get_loc('l')
    h_col = df.columns.get_loc('h')
    d_col = df.columns.get_loc('delta')
    df['resistance_lookup'] = 0
    df['resistance_indicator'] = 0

    lookup_col = df.columns.get_loc('resistance_lookup')
    new_col = df.columns.get_loc('resistance_indicator')

    reversal_col = df.columns.get_loc('c')   

    arr = df.values
    min_count = 0

    print('STOP RANGE:',stop_range)
    c = 0 

    for i in range(lookup_range,arr.shape[0]):
       # try:
        #DEFINE SUPPORT/RESISTANCE as the MIN or MAX of a lookup range
        max_ = max(arr[i - lookup_range : i + lookup_range2,reversal_col])

        # If the current val is at that reversal point:
        # Check and make sure we have an adequate movement off of that reversal point
        # this is defined by the pip_range and pip threshold, pip threshold is a user defined parameter
        if arr[i,reversal_col] == max_ and (i + lookup_range2 + stop_range < arr.shape[0]):
            lookup_ind = i
            val = arr[i,reversal_col]
            lookup_check = 0
            c = c + 1
            # Now that all of these conditions are met
            # We need to look into the future to see if we hit this reversal level, this is defined by the stop range
            for j in range(i + lookup_range2,i + lookup_range2 + stop_range):
                #If all of the following conditions are met, we will mark the array with a 1 signal to indicated we would place a trade                       
                #CANDLE BULL OR BEAR
                if arr[j,c_col] >= arr[j,o_col]:
                    bull = True
                else:
                    bull = False                        
                lower_bound = val - bound_pips
                upper_bound = val + bound_pips
                if i + lookup_range2 == j:
                    max_val = arr[j, reversal_col]
                else:
                    max_val = max(arr[i + lookup_range2 : j, reversal_col])
                cur_val = arr[j - 1, reversal_col]

                if lookup_check == 0 and \
                bull == False and\
                arr[j,d_col] >= candle_size and\
                max_val <= upper_bound and \
                cur_val <= upper_bound and cur_val >= lower_bound:
                    arr[j,new_col] = 1
                    arr[j,lookup_col] = lookup_ind
                    lookup_check = 1
                    break

       # except:
        #    pass

    print('COUNT:',c)

    e = time.time()
    print('TOTAL FUNCTION TIME:',(e-s)/60,' MINUTES')
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['resistance_indicator'] == 1].shape)
    return df





def run_single_pair_sim(pair,
                lookup_range,
                lookup_range2,
                stop_range,
                candle_size,
                bound_pips,
                sls,
                tps,
                spread):
    
    def strategy_tester_buy(df,col = 'support_indicator',sl = .1,tp = .5,num_trades = 300,risk_size = .01,entry = 'c',pr = True,spread = .01):
        def simulate_results(
            num_trades = 200,
            acc_val = 10000,
            rr = 6,
            risk_size = .02,
            win_rate = .2,
            pr = True):

            start = acc_val
            for i in range(num_trades):
                rand = random.random()
                if rand > win_rate:
                    acc_val = acc_val - (acc_val * risk_size)
                else:
                    acc_val = acc_val + (acc_val * (risk_size * rr))
            if pr:
                print('STARTING VAL:',round(start),' ENDING VAL:',round(acc_val))
            return round(acc_val)

        col_num = df.columns.get_loc(col)
        h_col = df.columns.get_loc('h')
        l_col = df.columns.get_loc('l')
        c_col = df.columns.get_loc('c')
        entry_col = df.columns.get_loc(entry)
        arr = df.values
        temp_df = df[df[col] == 1]

        trade_list = []
        trade_res = []
        wins = []
        losses = []
        for i in range(arr.shape[0]):

            if arr[i,col_num] == 1:
                val = arr[i,entry_col]
                trade_id = 0
                try:
                    for j in range(i + 1,i + 10000):
                        if i not in trade_list:
                            if trade_id == 0:
                                if arr[j,l_col] <= val - sl + spread:
                                    trade_res.append(-10)
                                    trade_list.append(i)
                                    losses.append(i)
                                    trade_id = 1

                                    break
                                if arr[j,h_col] >= val + tp:
                                    trade_res.append(50)
                                    trade_list.append(i)
                                    wins.append(i)
                                    trade_id = 1
                                    break
                except:
                    pass
        win_rate = len(wins) / len(trade_list)
        rr = (tp - spread) / (sl + spread)
       # num_trades = len(trade_list)
      #  print('WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size)
        vals = []


        acc_val = 10000



        for i in range(30):

            final_val = simulate_results(num_trades = num_trades,acc_val = acc_val,rr = rr,risk_size = risk_size,win_rate = win_rate,pr = pr)
            vals.append(final_val)

        return np.median(vals),win_rate,wins,losses

    def strategy_tester_sell(df,col = 'support_indicator',sl = .1,tp = .5,num_trades = 300,risk_size = .01,entry = 'c',pr = True,spread = .01):
        def simulate_results(
            num_trades = 200,
            acc_val = 10000,
            rr = 6,
            risk_size = .02,
            win_rate = .2,
            pr = True):

            start = acc_val
            for i in range(num_trades):
                rand = random.random()
                if rand > win_rate:
                    acc_val = acc_val - (acc_val * risk_size)
                else:
                    acc_val = acc_val + (acc_val * (risk_size * rr))
            if pr:
                print('STARTING VAL:',round(start),' ENDING VAL:',round(acc_val))
            return round(acc_val)

        col_num = df.columns.get_loc(col)
        h_col = df.columns.get_loc('h')
        l_col = df.columns.get_loc('l')
        c_col = df.columns.get_loc('c')
        entry_col = df.columns.get_loc(entry)
        arr = df.values
        temp_df = df[df[col] == 1]

        trade_list = []
        trade_res = []
        wins = []
        losses = []
        for i in range(arr.shape[0]):

            if arr[i,col_num] == 1:
                val = arr[i,entry_col]
                try:
                    for j in range(i + 1,i + 10000):
                        if i not in trade_list:
                            if arr[j,h_col] >= val + sl - spread:
                                trade_res.append(-10)
                                trade_list.append(i)
                                losses.append(i)
                                break
                            if arr[j,l_col] <= val - tp:
                                trade_res.append(50)
                                trade_list.append(i)
                                wins.append(i)
                                break
                except:
                    pass
        win_rate = len(wins) / len(trade_list)
        rr = (tp - spread) / (sl + spread)
       # num_trades = len(trade_list)
      #  print('WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size)
        vals = []


        acc_val = 10000



        for i in range(30):

            final_val = simulate_results(num_trades = num_trades,acc_val = acc_val,rr = rr,risk_size = risk_size,win_rate = win_rate,pr = pr)
            vals.append(final_val)

        return np.median(vals),win_rate,wins,losses


    s = time.time()
    df = load_df(pair = pair,granularity ='M5',start = datetime(2016,1,1,0,0,0),end = datetime(2022,7,31,0,0,0))
    print(df.shape)
    df_ = df.copy()

    print('RUNNING SINGLE PARAMETER LOOP FOR PARAMS:',
          'INSTRUMENT:',pair,
          'lookup_range',lookup_range,
          'lookup_range2', lookup_range2,
                    'stop_range', stop_range,
                    'candle_size' , candle_size,
                    'bound_pips' , bound_pips)

    df = get_support(df,
                    lookup_range = lookup_range,
                    lookup_range2 = lookup_range2,
                    stop_range = stop_range,
                    candle_size = candle_size,
                    bound_pips = bound_pips
                    )

    df_ = get_resistance(df_,
                    lookup_range = lookup_range,
                    lookup_range2 = lookup_range2,
                    stop_range = stop_range,
                    candle_size = candle_size,
                    bound_pips = bound_pips
                   )


    cols = ['instrument',
           'buy_support_val',
           'sell_resistance_val',
           'buy_resistance_val',
           'sell_support_val',
            'max_ending_val',
            'sl',
            'tp',
            'RR',
           'win_rate1',
           'win_rate2',
           'win_rate3',
           'win_rate4',
            'lookup_range',
            'lookup_range2',
            'stop_range',
            'candle_size',
            'bound_pips',
            'support_number_of_trades',
            'resistance_number_of_trades'
          ]

    trade_strategy = 'support_indicator'
    trade_strategy2 = 'resistance_indicator'

    #trade_strategy = 'resistance_indicatorc'
    num_trades = 400
    risk_size = .01

    starting_val = 10000
    instrument = pair
    timeframe = 'M5'
    temp_lst = []
    actual_number_of_trades = df[df[trade_strategy] == 1].shape[0]
    actual_number_of_trades2 = df_[df_[trade_strategy2] == 1].shape[0]


    for sl in sls:
        for tp in tps:
            if actual_number_of_trades == 0:
                ending_val,win_rate,wins,losses = 0,0,0,0
                ending_val4,win_rate4,wins4,losses4 = 0,0,0,0
            else:
                ending_val,win_rate,wins,losses = strategy_tester_buy(df,col = trade_strategy,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False,spread = spread)
                ending_val4,win_rate4,wins4,losses4 = strategy_tester_sell(df,col = trade_strategy,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False,spread = spread)   
                
            if actual_number_of_trades2 == 0:
                ending_val2,win_rate2,wins2,losses2 = 0,0,0,0
                ending_val3,win_rate3,wins3,losses3 = 0,0,0,0
            else:
                ending_val2,win_rate2,wins2,losses2 = strategy_tester_sell(df_,col = trade_strategy2,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False,spread = spread)
                ending_val3,win_rate3,wins3,losses3 = strategy_tester_buy(df_,col = trade_strategy2,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False,spread = spread) 
                
                
            max_ending_val = max(ending_val,
                   ending_val2,
                   ending_val3,
                   ending_val4)
            lst = [instrument,
                   ending_val,
                   ending_val2,
                   ending_val3,
                   ending_val4,
                   max_ending_val,
                    sl,
                    tp,
                    round(tp / sl,4),
                   round(win_rate,4),
                   round(win_rate2,4),
                   round(win_rate3,4),
                   round(win_rate4,4),
                    lookup_range,
                    lookup_range2,
                    stop_range,
                    candle_size,
                    bound_pips,
                   actual_number_of_trades,
                   actual_number_of_trades2
                  ]
            temp_lst.append(lst)

    final_df = pd.DataFrame(temp_lst,columns = cols)
    final_df = final_df.sort_values(by = 'max_ending_val',ascending = False)
    
    e = time.time()
    best_buy_support = final_df['buy_support_val'].max()
    best_sell_resistance = final_df['sell_resistance_val'].max()
    best_buy_resistance = final_df['buy_resistance_val'].max()
    best_sell_support = final_df['sell_support_val'].max()
    max_val_all = final_df['max_ending_val'].max()
    print('MAX VAL:',round(max_val_all),'||best buy support:',round(best_buy_support),'||best sell resistance:',
          round(best_sell_resistance),'||best buy resistance:',round(best_buy_resistance),'||best sell support:',round(best_sell_support))


    print('TOTAL SINGLE PAIR SIM TIME:',round((e-s)/60,1),'MINUTES')
    return final_df

def run_all_pairs_sim_loop(
                    lookup_range,
                    lookup_range2,
                    stop_range,
                    candle_size ,
                    bound_pips,
                    spread):
    dir_name = os.getcwd() + '/reversal_counter_sim_results/' + str(lookup_range) + '_' + str(lookup_range2) + '_' + str(stop_range) + '_' + str(candle_size) + '_' + str(bound_pips)
    print(dir_name)
    isdir = os.path.isdir(dir_name) 
    if isdir:
        pass
    else:
        print('MAKING DIRECTORY:',dir_name)
        os.mkdir(dir_name) 

    if len(os.listdir(dir_name)) > 0:
        print('file already exists in this directory, passing...')
    else:
        print('file does not exist, running sim')


        all_pairs=['GBP_JPY','EUR_USD','USD_JPY','EUR_JPY','AUD_JPY','CAD_JPY','CHF_JPY',
                   'GBP_USD','USD_CAD','USD_CHF','AUD_USD',
                   'NZD_USD','AUD_CAD','AUD_CHF','AUD_NZD','CAD_CHF',
                   'EUR_AUD','EUR_CAD','EUR_CHF','NZD_CAD','NZD_CHF'] 


        final_df = pd.DataFrame([0])    
        for pair in all_pairs:
            print(pair)  

            if 'JPY' in pair:
                sls = [.05,.075,.1,.125,.15,.2,.25]
                tps = [.1,.15,.2,.25,.3,.4,.5,.7,.9,1.1,1.3,1.5]
                spread = .01
                candle_size_ = candle_size * 100
                bound_pips_ = bound_pips * 100
            else:
                sls = [.0005,.00075,.001,.00125,.0015,.002,.0025]
                tps = [.001,.0015,.002,.0025,.003,.004,.005,.007,.009,.011,.013,.015]
                spread = .0001
                candle_size_ = candle_size 
                bound_pips_ = bound_pips        

            df = run_single_pair_sim(pair,
                            lookup_range,
                            lookup_range2,
                            stop_range,
                            candle_size_,
                            bound_pips_,
                            sls,
                            tps,
                            spread)    
            if final_df.shape[0] == 1:
                final_df = df
            else:
                final_df = pd.concat([final_df,df],axis = 0)

        final_df = final_df.sort_values(by = 'max_ending_val',ascending = False)    

        gb = final_df.groupby('instrument').agg({'max_ending_val':max,'buy_support_val':max,'sell_resistance_val':max,'buy_resistance_val':max,'sell_support_val':max})
        gb = gb.reset_index()
        gb = gb.sort_values(by = 'max_ending_val',ascending = False)    
        for i in range(gb.shape[0]):
            print('pair',gb['instrument'].iloc[i],'||MAX VAL',gb['max_ending_val'].iloc[i],'||best buy support:',gb['buy_support_val'].iloc[i],'||best sell resistance:',
                  gb['sell_resistance_val'].iloc[i],'||best buy resistance:',gb['buy_resistance_val'].iloc[i],'||best sell support:',gb['sell_support_val'].iloc[i])

        print('MEDIANS:',np.median(gb['max_ending_val']),np.median(gb['buy_support_val']),np.median(gb['sell_resistance_val']),np.median(gb['buy_resistance_val']),np.median(gb['sell_support_val']))

        path = dir_name + '/' + str(round(np.median(gb['max_ending_val']))) + '_' + str(round(np.median(gb['buy_support_val']))) + '_' + str(round(np.median(gb['sell_resistance_val']))) + '_' + str(round(np.median(gb['buy_resistance_val']))) + '_' + str(round(np.median(gb['sell_support_val'])))
        path = path + '.csv'
        print('FINAL PATH:',path)
        print('SAVING DATA')
        final_df.to_csv(path,index = False)
        print()
        print()
        print()
        print()
        print()
        


In [4]:
from datetime import timezone
now = datetime.now(timezone.utc)
now.year


2023

In [5]:
now.weekday()

0

In [185]:
def get_support(df,
                lookup_range = 1500,
                stop_range = 2200,
                bound_pips = .001,
                bound_pips2 = .005
               ):
    print('GETTING SUPPORT INDICATOR')
    s = time.time()
    c_col = df.columns.get_loc('c')
    o_col = df.columns.get_loc('o')
    l_col = df.columns.get_loc('l')
    h_col = df.columns.get_loc('h')
    d_col = df.columns.get_loc('delta')
    df['support_lookup'] = 0
    df['support_indicator'] = 0

    lookup_col = df.columns.get_loc('support_lookup')
    new_col = df.columns.get_loc('support_indicator')

    reversal_col = df.columns.get_loc('c')   

    arr = df.values
    min_count = 0

    print('STOP RANGE:',stop_range)
    c = 0 

    for i in range(lookup_range,arr.shape[0]):
       # try:
        #DEFINE SUPPORT/RESISTANCE as the MIN or MAX of a lookup range
        #min_ = min(arr[i - lookup_range : i + lookup_range2,reversal_col])
        min_ = min(arr[i - lookup_range : i + 1,reversal_col])

        
        reversal_ind = 0
        if arr[i,reversal_col] == min_:
            for k in range(i,i + stop_range):
                if k >= arr.shape[0]:
                    ind = 0
                    break
                if arr[k,reversal_col] < min_:
                    break
                if arr[k,reversal_col] - min_ >= bound_pips2 :
                    reversal_ind = 1
                    ind = k
                    break
                
        # If the current val is at that reversal point:
        # Check and make sure we have an adequate movement off of that reversal point
        # this is defined by the pip_range and pip threshold, pip threshold is a user defined parameter
        if reversal_ind == 1 and ind != 0:
            lookup_ind = i
            val = arr[i,reversal_col]
            lookup_check = 0
            # Now that all of these conditions are met
            # We need to look into the future to see if we hit this reversal level, this is defined by the stop range
            if ind + stop_range < df.shape[0]:
                for j in range(ind,ind + stop_range):

                    lower_bound = val - bound_pips 
                    upper_bound = val + bound_pips                    
                    cur_val = arr[j, reversal_col]
                    max_val = max(arr[j:j + 1000, reversal_col])
                    second_delta = max_val - val
                

                    if cur_val < lower_bound and arr[j + 24,reversal_col] < lower_bound:
                        lookup_check = 1

                    if lookup_check == 0 and \
                    cur_val <= upper_bound and \
                    cur_val >= lower_bound and \
                    second_delta >= bound_pips2 and \
                    arr[j + 30,reversal_col] > val:
                        if j + 30 + (stop_range * 3) >= df.shape[0]:
                            loop_val = df.shape[0]
                        else:
                            loop_val = j + 30 + (stop_range * 3)
                        for k in range(j + 30, loop_val):
                            cur_val2 = arr[k, reversal_col]
                            
                            if lookup_check == 0 and \
                            cur_val2 <= val and \
                            cur_val2 >= lower_bound and \
                            max(arr[j:k,reversal_col]) - val >= bound_pips2:                     
                                arr[k,new_col] = 1
                                arr[k,lookup_col] = lookup_ind
                                lookup_check = 1
                                break

    e = time.time()
    print('TOTAL FUNCTION TIME:',(e-s)/60,' MINUTES')
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['support_indicator'] == 1].shape)
    return df

def get_resistance(df,
                lookup_range = 1500,
                lookup_range2 = 2400,
                stop_range = 2200,
                candle_size = .001,
                bound_pips = .001
               ):
    print('GETTING RESISTANCE INDICATOR')
    s = time.time()
    c_col = df.columns.get_loc('c')
    o_col = df.columns.get_loc('o')
    l_col = df.columns.get_loc('l')
    h_col = df.columns.get_loc('h')
    d_col = df.columns.get_loc('delta')
    df['resistance_lookup'] = 0
    df['resistance_indicator'] = 0

    lookup_col = df.columns.get_loc('resistance_lookup')
    new_col = df.columns.get_loc('resistance_indicator')

    reversal_col = df.columns.get_loc('c')   

    arr = df.values
    min_count = 0

    print('STOP RANGE:',stop_range)


    for i in range(lookup_range,arr.shape[0]):
       # try:
        #DEFINE SUPPORT/RESISTANCE as the MIN or MAX of a lookup range
        max_ = max(arr[i - lookup_range : i + lookup_range2,reversal_col])

        # If the current val is at that reversal point:
        # Check and make sure we have an adequate movement off of that reversal point
        # this is defined by the pip_range and pip threshold, pip threshold is a user defined parameter
        if arr[i,reversal_col] == max_ and (i + lookup_range2 + stop_range < arr.shape[0]):
            lookup_ind = i
            val = arr[i,reversal_col]
            lookup_check = 0
            c = c + 1
            # Now that all of these conditions are met
            # We need to look into the future to see if we hit this reversal level, this is defined by the stop range
            for j in range(i + lookup_range2,i + lookup_range2 + stop_range):
                #If all of the following conditions are met, we will mark the array with a 1 signal to indicated we would place a trade                       
                #CANDLE BULL OR BEAR
                if arr[j,c_col] >= arr[j,o_col]:
                    bull = True
                else:
                    bull = False                        
                lower_bound = val - bound_pips
                upper_bound = val + bound_pips
                if i + lookup_range2 == j:
                    max_val = arr[j, reversal_col]
                else:
                    max_val = max(arr[i + lookup_range2 : j, reversal_col])
                cur_val = arr[j - 1, reversal_col]

                if lookup_check == 0 and \
                bull == False and\
                arr[j,d_col] >= candle_size and\
                max_val <= upper_bound and \
                cur_val <= upper_bound and cur_val >= lower_bound:
                    arr[j,new_col] = 1
                    arr[j,lookup_col] = lookup_ind
                    lookup_check = 1
                    break


    e = time.time()
    print('TOTAL FUNCTION TIME:',(e-s)/60,' MINUTES')
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['resistance_indicator'] == 1].shape)
    return df



pair = 'EUR_USD'
df = load_df(pair = pair,granularity ='M5',start = datetime(2016,1,1,0,0,0),end = datetime(2022,7,31,0,0,0))
print(df.shape)
df = get_support(df,
                lookup_range = 300,
                stop_range = 10000,
                bound_pips = .001,
                bound_pips2 = .005
               )
print(df.shape)
df.head()


PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/EUR_USD_M5/2016-01-01_2022-07-31.csv
(490023, 25)
(490023, 25)
GETTING SUPPORT INDICATOR
STOP RANGE: 10000


KeyboardInterrupt: 

In [250]:

def load_df(pair = 'EUR_USD'
            ,granularity = 'M5'
            ,start = datetime(2016,1,1,0,0,0)
            ,end = datetime(2022,7,31,0,0,0)):
    def convert_timestamp(df):
        time_col = df.columns.get_loc('time')
        df['minute_col'] = 0
        new_col = df.columns.get_loc('minute_col')
        arr = df.values 
        for i in range(arr.shape[0]):
            arr[i,new_col] = pd.Timestamp(arr[i,time_col]).minute
        return pd.DataFrame(arr,columns = df.columns)
    dir_name = os.getcwd() + '/' + pair + '_' + granularity
    path = dir_name + '/' + str(date(start.year,start.month,start.day)) + '_' + str(date(end.year,end.month,end.day)) + '.csv'
    print('PATH:',path)
    df = pd.read_csv(path)
    df = add_delta_cols(df)
    df = convert_timestamp(df)
    print(df.shape)
    return df


def get_support(df,
                lookup_range = 1500,
                pip_delta = .001
               ):
    print('GETTING SUPPORT INDICATOR')
    print('LOOKUP RANGE AND PIP DELTA:',lookup_range,pip_delta)
    s = time.time()
    c_col = df.columns.get_loc('c')
    o_col = df.columns.get_loc('o')
    l_col = df.columns.get_loc('l')
    h_col = df.columns.get_loc('h')
    d_col = df.columns.get_loc('delta')
    minute_col = df.columns.get_loc('minute_col')
    df['support_indicator'] = 0
    new_col = df.columns.get_loc('support_indicator')
    arr = df.values
    min_count = 0
    c = 0 
    if pip_delta < .01:
        low_spread = 0 #.0001
    else:
        low_spread = 0 #.01
    for i in range(max(lookup_range,12),arr.shape[0]):
       # try:
        #DEFINE SUPPORT/RESISTANCE as the MIN or MAX of a lookup range
        df.iloc[i - 11 : i + 1,:]
        if arr[i,c_col] <= min(arr[i - lookup_range : i,c_col]) and arr[i,c_col] <= min(arr[i - 11 : i + 1,l_col]) + low_spread  and (arr[i - 11,o_col] - arr[i,c_col]) >= pip_delta and arr[i,minute_col] == 55:
            arr[i,new_col] = 1

    print('COUNT:',c)

    e = time.time()
    print('TOTAL FUNCTION TIME:',(e-s)/60,' MINUTES')
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['support_indicator'] == 1].shape)
    return df

def get_support(df,
                lookup_range = 1500,
                pip_delta = .001
               ):
    print('GETTING SUPPORT INDICATOR')
    print('LOOKUP RANGE AND PIP DELTA:',lookup_range,pip_delta)
    s = time.time()
    c_col = df.columns.get_loc('c')
    o_col = df.columns.get_loc('o')
    l_col = df.columns.get_loc('l')
    h_col = df.columns.get_loc('h')
    d_col = df.columns.get_loc('delta')
    minute_col = df.columns.get_loc('minute_col')
    df['support_indicator'] = 0
    new_col = df.columns.get_loc('support_indicator')
    arr = df.values
    min_count = 0
    c = 0 
    if pip_delta < .01:
        low_spread = 0 #.0001
    else:
        low_spread = 0 #.01
    for i in range(max(lookup_range,12),arr.shape[0]):
       # try:
        #DEFINE SUPPORT/RESISTANCE as the MIN or MAX of a lookup range
        if arr[i,h_col] == arr[i,o_col] and arr[i,l_col] == arr[i,c_col] and arr[i,o_col] - arr[i,c_col] >= pip_delta:
        #if arr[i,c_col] <= min(arr[i - lookup_range : i,c_col]) and arr[i,c_col] <= min(arr[i - 11 : i + 1,l_col]) + low_spread  and (arr[i - 11,o_col] - arr[i,c_col]) >= pip_delta and arr[i,minute_col] == 55:
            arr[i,new_col] = 1

    print('COUNT:',c)

    e = time.time()
    print('TOTAL FUNCTION TIME:',(e-s)/60,' MINUTES')
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['support_indicator'] == 1].shape)
    return df

def get_support(df,
                lookup_range = 1500,
                pip_delta = .001
               ):
    print('GETTING SUPPORT INDICATOR')
    print('LOOKUP RANGE AND PIP DELTA:',lookup_range,pip_delta)
    s = time.time()
    c_col = df.columns.get_loc('c')
    o_col = df.columns.get_loc('o')
    l_col = df.columns.get_loc('l')
    h_col = df.columns.get_loc('h')
    d_col = df.columns.get_loc('delta')
    minute_col = df.columns.get_loc('minute_col')
    df['support_indicator'] = 0
    new_col = df.columns.get_loc('support_indicator')
    arr = df.values
    min_count = 0
    c = 0 
    high_spread = .0002
    low_spread = .0001
    if pip_delta < .01:
        low_spread = 0 #.0001
    else:
        low_spread = 0 #.01
    for i in range(max(lookup_range,48),arr.shape[0]):
       # try:
        #DEFINE SUPPORT/RESISTANCE as the MIN or MAX of a lookup range

        if arr[i,c_col] <= min(arr[i - 47 : i + 1,l_col]) + low_spread and arr[i - 47,o_col] >= max(arr[i - 47 : i + 1,h_col]) - high_spread  and (arr[i - 47,o_col] - arr[i,c_col]) >= pip_delta and arr[i,minute_col] == 55:
            arr[i,new_col] = 1

    print('COUNT:',c)

    e = time.time()
    print('TOTAL FUNCTION TIME:',(e-s)/60,' MINUTES')
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['support_indicator'] == 1].shape)
    return df


def get_support(df,
                lookup_range = 1500,
                pip_delta = .001,
                time_range = 11,
                high_spread = .0005,
                low_spread = 0
               ):
    print('GETTING SUPPORT INDICATOR')
    print('LOOKUP RANGE AND PIP DELTA:',lookup_range,pip_delta)
    s = time.time()
    c_col = df.columns.get_loc('c')
    o_col = df.columns.get_loc('o')
    l_col = df.columns.get_loc('l')
    h_col = df.columns.get_loc('h')
    d_col = df.columns.get_loc('delta')
    minute_col = df.columns.get_loc('minute_col')
    df['support_indicator'] = 0
    new_col = df.columns.get_loc('support_indicator')
    arr = df.values
    min_count = 0
    c = 0 
   # high_spread = .0003
    #low_spread = .0001
    #low_spread = 0
    #high_spread = .0005
    
    for i in range(max(lookup_range,time_range + 1),arr.shape[0]):
       # try:
        #DEFINE SUPPORT/RESISTANCE as the MIN or MAX of a lookup range

        if arr[i,c_col] <= min(arr[i - time_range : i + 1,l_col]) + low_spread and arr[i - time_range,o_col] >= max(arr[i - time_range : i + 1,h_col]) - high_spread  and (arr[i - time_range,o_col] - arr[i,c_col]) >= pip_delta and arr[i,minute_col] == 55:
            arr[i,new_col] = 1

    print('COUNT:',c)

    e = time.time()
    print('TOTAL FUNCTION TIME:',(e-s)/60,' MINUTES')
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['support_indicator'] == 1].shape)
    return df

def get_support(df,
                lookup_range = 1500,
                pip_delta = .001,
                time_range = 11,
                high_spread = .0005,
                low_spread = 0
               ):
    print('GETTING SUPPORT INDICATOR')
    print('LOOKUP RANGE AND PIP DELTA:',lookup_range,pip_delta)
    s = time.time()
    c_col = df.columns.get_loc('c')
    o_col = df.columns.get_loc('o')
    l_col = df.columns.get_loc('l')
    h_col = df.columns.get_loc('h')
    d_col = df.columns.get_loc('delta')
    minute_col = df.columns.get_loc('minute_col')
    df['support_indicator'] = 0
    new_col = df.columns.get_loc('support_indicator')
    arr = df.values
    min_count = 0
    c = 0 
   # high_spread = .0003
    #low_spread = .0001
    #low_spread = 0
    #high_spread = .0005
    
    for i in range(max(lookup_range,time_range + 1),arr.shape[0]):
       # try:
        #DEFINE SUPPORT/RESISTANCE as the MIN or MAX of a lookup range

        if (arr[i - time_range,o_col] - arr[i,c_col]) >= pip_delta and arr[i,minute_col] == 55:
            arr[i,new_col] = 1

    print('COUNT:',c)

    e = time.time()
    print('TOTAL FUNCTION TIME:',(e-s)/60,' MINUTES')
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['support_indicator'] == 1].shape)
    return df

all_pairs=['GBP_JPY','EUR_USD','USD_JPY','EUR_JPY','AUD_JPY','CAD_JPY','CHF_JPY',
           'GBP_USD','USD_CAD','USD_CHF','AUD_USD',
           'NZD_USD','AUD_CAD','AUD_CHF','AUD_NZD','CAD_CHF',
           'EUR_AUD','EUR_CAD','EUR_CHF','NZD_CAD','NZD_CHF'] 


sls = [.0005,.00075,.001,.00125,.0015,.002,.0025]
tps = [.001,.0015,.002,.0025,.003,.004,.005,.007,.009,.011,.013,.015]
spread = .0001
high_spread = .002
low_spread = 0

df = load_df(pair = pair,granularity ='M5',start = datetime(2016,1,1,0,0,0),end = datetime(2022,7,31,0,0,0))
df = get_support(df,
                lookup_range = 2,
                pip_delta = .005,
                 time_range = 47,
                high_spread = high_spread,
                low_spread = low_spread
               )
print(df.shape)
for sl in sls:
    for tp in tps:

        val,win_rate,wins,losses = strategy_tester_buy(df,col = 'support_indicator',sl = sl,tp = tp,num_trades = 400,risk_size = .01,entry = 'c',pr = False,spread = spread)
        print(sl,tp,val)


PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/CAD_JPY_M5/2016-01-01_2022-07-31.csv
(491481, 26)
GETTING SUPPORT INDICATOR
LOOKUP RANGE AND PIP DELTA: 2 0.005
COUNT: 0
TOTAL FUNCTION TIME: 0.0074365496635437015  MINUTES
SHAPE (19192, 27)
(491481, 27)
0.0005 0.001 409.0
0.0005 0.0015 591.0
0.0005 0.002 808.5
0.0005 0.0025 1058.0
0.0005 0.003 1410.0
0.0005 0.004 2488.0
0.0005 0.005 4166.5
0.0005 0.007 10229.0
0.0005 0.009 32302.0
0.0005 0.011 67698.5
0.0005 0.013 211260.5
0.0005 0.015 757027.0
0.00075 0.001 384.0
0.00075 0.0015 452.0
0.00075 0.002 562.0
0.00075 0.0025 676.0
0.00075 0.003 862.0
0.00075 0.004 1261.5
0.00075 0.005 1699.0
0.00075 0.007 4477.5
0.00075 0.009 7894.5
0.00075 0.011 11764.0
0.00075 0.013 36268.5
0.00075 0.015 67037.5
0.001 0.001 333.0
0.001 0.0015 406.5
0.001 0.002 511.0
0.001 0.0025 552.0
0.001 0.003 658.0
0.001 0.004 827.0
0.001 0.005 1205.5
0.001 0.007 2146.0
0.001 0.009 3722.5
0.001 0.011 6275.0
0.001 0.013 11666.5
0.001 0.015 17762.5
0.001

In [252]:
def load_df(pair = 'EUR_USD'
            ,granularity = 'M5'
            ,start = datetime(2016,1,1,0,0,0)
            ,end = datetime(2022,7,31,0,0,0)):
    def convert_timestamp(df):
        time_col = df.columns.get_loc('time')
        df['minute_col'] = 0
        new_col = df.columns.get_loc('minute_col')
        arr = df.values 
        for i in range(arr.shape[0]):
            arr[i,new_col] = pd.Timestamp(arr[i,time_col]).minute
        return pd.DataFrame(arr,columns = df.columns)
    dir_name = os.getcwd() + '/' + pair + '_' + granularity
    path = dir_name + '/' + str(date(start.year,start.month,start.day)) + '_' + str(date(end.year,end.month,end.day)) + '.csv'
    print('PATH:',path)
    df = pd.read_csv(path)
    df = add_delta_cols(df)
    df = convert_timestamp(df)
    print(df.shape)
    return df

df = load_df(pair = pair,granularity ='M5',start = datetime(2016,1,1,0,0,0),end = datetime(2022,7,31,0,0,0))
df2 = load_df(pair = pair,granularity ='H4',start = datetime(2016,1,1,0,0,0),end = datetime(2022,7,31,0,0,0))
print(df2.shape)




PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/NZD_CHF_M5/2016-01-01_2022-07-31.csv
(490624, 26)
PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/NZD_CHF_H4/2016-01-01_2022-07-31.csv
(10672, 14)
(10672, 14)


In [253]:
df2.head()

,time,volume,o,h,l,c,utc_timestamp,est_timestamp,delta_vector,delta_max,upper_wick,lower_wick,delta,minute_col
0,2016-01-03T22:00:00.000000000Z,7404,0.68374,0.68374,0.67678,0.67683,2016-01-03 22:00:00+00:00,2016-01-03 17:00:00-05:00,-0.00691,0.00696,0.0,0.00005,0.00691,0
1,2016-01-04T02:00:00.000000000Z,5726,0.67688,0.677,0.67132,0.67168,2016-01-04 02:00:00+00:00,2016-01-03 21:00:00-05:00,-0.0052,0.00568,0.00012,0.00036,0.0052,0
2,2016-01-04T06:00:00.000000000Z,8797,0.67166,0.6739,0.67166,0.67246,2016-01-04 06:00:00+00:00,2016-01-04 01:00:00-05:00,0.0008,0.00224,0.00144,0.0,0.0008,0
3,2016-01-04T10:00:00.000000000Z,7013,0.67246,0.67707,0.67208,0.67578,2016-01-04 10:00:00+00:00,2016-01-04 05:00:00-05:00,0.00332,0.00499,0.00129,0.00038,0.00332,0
4,2016-01-04T14:00:00.000000000Z,9421,0.6758,0.67858,0.67514,0.67678,2016-01-04 14:00:00+00:00,2016-01-04 09:00:00-05:00,0.00098,0.00344,0.0018,0.00066,0.00098,0


In [254]:
np.percentile(df2['delta'],90)

0.0023999999999999577

In [260]:
pd.DataFrame(df2.median()).T

,next_30,next_hour,next_two_hours,next_four_hours,next_eight_hours
0,1.6,1.6,0.75,0.1,3.9


In [247]:
def test_future(df,pair,column = 'support_indicator'):
    if 'JPY' in pair:
        mult = 100
    else:
        mult = 100 * 100
    c_col = df.columns.get_loc('c')
    col = df.columns.get_loc(column)
    
    lst = []
    arr = df.values
    for i in range(arr.shape[0]):
        if arr[i,col] == 1:
            c = arr[i,c_col]
            lst.append([mult * (arr[i + 6,c_col] - c),
                        mult * (arr[i + 12,c_col] - c),
                        mult * (arr[i + 24,c_col] - c),
                        mult * (arr[i + 48,c_col] - c),
                        mult * (arr[i + 100,c_col] - c)                       
                       ])
    df2 = pd.DataFrame(lst,columns = ['next_30','next_hour','next_two_hours','next_four_hours','next_eight_hours'])  
    print('MEDIAN:')
    print(df2.median())
    print()
    print('MEAN:')
    print(df2.mean())
    print()
    print('SUM:')
    print(print(pd.DataFrame(df2.sum()).T))
    
    return df2


df2 = test_future(df,pair)
print(df2.shape)
df2.head()

MEDIAN:
next_30             0.4
next_hour           0.9
next_two_hours      2.0
next_four_hours     1.8
next_eight_hours    5.7
dtype: float64

MEAN:
next_30             0.372085
next_hour           0.740636
next_two_hours      1.746643
next_four_hours     2.896466
next_eight_hours    6.077385
dtype: float64

SUM:
   next_30  next_hour  next_two_hours  next_four_hours  next_eight_hours
0    105.3      209.6           494.3            819.7            1719.9
None
(283, 5)


,next_30,next_hour,next_two_hours,next_four_hours,next_eight_hours
0,-0.5,-12.3,-16.1,-25.1,3.5
1,-3.9,-3.8,-29.4,-10.5,-16.0
2,-5.2,-25.6,-9.0,-14.6,4.0
3,10.1,16.6,18.9,7.8,40.0
4,-3.4,-11.6,14.2,12.7,-2.4


In [246]:
.01 * 100

1.0

In [245]:
.0001 * 10000

1.0

In [238]:
print(pd.DataFrame(df2.sum()).T)

   next_30  next_hour  next_two_hours  next_four_hours  next_eight_hours
0  0.01053    0.02096         0.04943          0.08197           0.17199


In [234]:
df2.mean()

next_30             0.000037
next_hour           0.000074
next_two_hours      0.000175
next_four_hours     0.000290
next_eight_hours    0.000608
dtype: float64

In [235]:
df2.median()

next_30             0.00004
next_hour           0.00009
next_two_hours      0.00020
next_four_hours     0.00018
next_eight_hours    0.00057
dtype: float64

In [264]:
pd.DataFrame(df2.sum(),columns = [pair]).T

,next_30,next_hour,next_two_hours,next_four_hours,next_eight_hours
NZD_CHF,-57.7,-6.2,-24.7,-122.0,33.6


In [282]:

def get_support(df,
                lookup_range = 1500,
                pip_delta = .001,
                time_range = 11,
                high_spread = .0005,
                low_spread = 0
               ):
    print('GETTING SUPPORT INDICATOR')
    print('LOOKUP RANGE AND PIP DELTA:',lookup_range,pip_delta)
    s = time.time()
    c_col = df.columns.get_loc('c')
    o_col = df.columns.get_loc('o')
    l_col = df.columns.get_loc('l')
    h_col = df.columns.get_loc('h')
    d_col = df.columns.get_loc('delta')
    minute_col = df.columns.get_loc('minute_col')
    df['support_indicator'] = 0
    new_col = df.columns.get_loc('support_indicator')
    arr = df.values
    min_count = 0
    c = 0 
   # high_spread = .0003
    #low_spread = .0001
    #low_spread = 0
    #high_spread = .0005
    
    for i in range(max(lookup_range,time_range + 1),arr.shape[0]):
       # try:
        #DEFINE SUPPORT/RESISTANCE as the MIN or MAX of a lookup range

        if (arr[i - time_range,o_col] - arr[i,c_col]) >= pip_delta and arr[i,minute_col] == 55:
            arr[i,new_col] = 1

    print('COUNT:',c)

    e = time.time()
    print('TOTAL FUNCTION TIME:',(e-s)/60,' MINUTES')
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['support_indicator'] == 1].shape)
    return df


def get_support(df,
                df2,
                lookup_range = 1500,
                pip_delta = .001,
                time_range = 11,
                high_spread = .0005,
                low_spread = 0,
                percentile = 98
               ):
    print('GETTING SUPPORT INDICATOR')
    print('LOOKUP RANGE AND PIP DELTA:',lookup_range,pip_delta)
    s = time.time()
    c_col = df.columns.get_loc('c')
    o_col = df.columns.get_loc('o')
    l_col = df.columns.get_loc('l')
    h_col = df.columns.get_loc('h')
    d_col = df.columns.get_loc('delta')
    minute_col = df.columns.get_loc('minute_col')
    df['support_indicator'] = 0
    new_col = df.columns.get_loc('support_indicator')
    arr = df.values
    min_count = 0
    c = 0 
   # high_spread = .0003
    #low_spread = .0001
    #low_spread = 0
    #high_spread = .0005
    min_filter = np.percentile(df2['delta'],percentile)
    
    for i in range(max(lookup_range,time_range + 1),arr.shape[0]):
       # try:
        #DEFINE SUPPORT/RESISTANCE as the MIN or MAX of a lookup range

        if (arr[i - time_range,o_col] - arr[i,c_col]) >= min_filter and arr[i,minute_col] == 55 and sum(arr[i - 250: i,new_col]) == 0:
            arr[i,new_col] = 1

    print('COUNT:',c)

    e = time.time()
    print('TOTAL FUNCTION TIME:',(e-s)/60,' MINUTES')
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['support_indicator'] == 1].shape)
    return df

def get_support(df,
                df2,
                lookup_range = 1500,
                pip_delta = .001,
                time_range = 11,
                high_spread = .0005,
                low_spread = 0,
                percentile = 98
               ):
    print('GETTING SUPPORT INDICATOR')
    print('LOOKUP RANGE AND PIP DELTA:',lookup_range,pip_delta)
    s = time.time()
    c_col = df.columns.get_loc('c')
    o_col = df.columns.get_loc('o')
    l_col = df.columns.get_loc('l')
    h_col = df.columns.get_loc('h')
    d_col = df.columns.get_loc('delta')
    minute_col = df.columns.get_loc('minute_col')
    df['support_indicator'] = 0
    new_col = df.columns.get_loc('support_indicator')
    arr = df.values
    min_count = 0
    c = 0 
   # high_spread = .0003
    #low_spread = .0001
    #low_spread = 0
    #high_spread = .0005
    min_filter = np.percentile(df2['delta'],percentile)
    
    for i in range(max(lookup_range,time_range + 1),arr.shape[0]):
       # try:
        #DEFINE SUPPORT/RESISTANCE as the MIN or MAX of a lookup range

        if (arr[i - time_range,o_col] - arr[i,c_col]) >= min_filter and arr[i,minute_col] == 55 and sum(arr[i - 50: i,new_col]) == 0:
            arr[i,new_col] = 1

    print('COUNT:',c)

    e = time.time()
    print('TOTAL FUNCTION TIME:',(e-s)/60,' MINUTES')
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['support_indicator'] == 1].shape)
    return df
def test_future(df,pair,column = 'support_indicator'):
    if 'JPY' in pair:
        mult = 100
    else:
        mult = 100 * 100
    c_col = df.columns.get_loc('c')
    col = df.columns.get_loc(column)
    
    lst = []
    arr = df.values
    for i in range(arr.shape[0] - 101):
        if arr[i,col] == 1:
            c = arr[i,c_col]

            lst.append([mult * (arr[i + 6,c_col] - c),
                        mult * (arr[i + 12,c_col] - c),
                        mult * (arr[i + 24,c_col] - c),
                        mult * (arr[i + 48,c_col] - c),
                        mult * (arr[i + 100,c_col] - c)                       
                       ])
    df2 = pd.DataFrame(lst,columns = ['next_30','next_hour','next_two_hours','next_four_hours','next_eight_hours'])  
    print('MEDIAN:')
    print(df2.median())
    print()
    print('MEAN:')
    print(df2.mean())
    print()
    print('SUM:')
    print(print(pd.DataFrame(df2.sum()).T))
    
    return pd.DataFrame(df2.median(),columns = [pair]).T,pd.DataFrame(df2.mean(),columns = [pair]).T,pd.DataFrame(df2.sum(),columns = [pair]).T



all_pairs=['GBP_JPY','EUR_USD','USD_JPY','EUR_JPY','AUD_JPY','CAD_JPY','CHF_JPY',
           'GBP_USD','USD_CAD','USD_CHF','AUD_USD',
           'NZD_USD','AUD_CAD','AUD_CHF','AUD_NZD','CAD_CHF',
           'EUR_AUD','EUR_CAD','EUR_CHF','NZD_CAD','NZD_CHF'] 




final_df = pd.DataFrame([0])
final_med = pd.DataFrame([0])
final_mean = pd.DataFrame([0])
final_sum = pd.DataFrame([0])
for pair in all_pairs:
    print(pair)  

    if 'JPY' in pair:
        sls = [.05,.075,.1,.125,.15,.2,.25]
        tps = [.1,.15,.2,.25,.3,.4,.5,.7,.9,1.1,1.3,1.5]
        spread = .01
        high_spread = .0005 * 100
        low_spread = 0 * 100
        pip_delta = .005 * 100
    else:
        sls = [.0005,.00075,.001,.00125,.0015,.002,.0025]
        tps = [.001,.0015,.002,.0025,.003,.004,.005,.007,.009,.011,.013,.015]
        spread = .0001
        high_spread = .0005
        low_spread = 0
        pip_delta = .005 

    df = load_df(pair = pair,granularity ='M5',start = datetime(2016,1,1,0,0,0),end = datetime(2022,7,31,0,0,0))
    df2 = load_df(pair = pair,granularity ='H4',start = datetime(2016,1,1,0,0,0),end = datetime(2022,7,31,0,0,0))
    df = get_support(df,
                     df2,
                lookup_range = 2,
                pip_delta = pip_delta,
                 time_range = 47,
                high_spread = high_spread,
                low_spread = low_spread,
                percentile = 99
               )
    meds,means,sums = test_future(df,pair)
    for j in range(meds.shape[1]):
        if meds.iloc[0,j] < .01:
            meds.iloc[0,j] = 0
            
    for j in range(means.shape[1]):
        if means.iloc[0,j] < .01:
            means.iloc[0,j] = 0  
            
    for j in range(sums.shape[1]):
        if sums.iloc[0,j] < .01:
            sums.iloc[0,j] = 0  
            
    if final_med.shape[1] == 1:
        final_med = meds
        final_mean = means
        final_sum = sums
    else:
        final_med = pd.concat([final_med,meds])
        final_mean = pd.concat([final_mean,means])
        final_sum = pd.concat([final_sum,sums])

final_med


GBP_JPY
PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/GBP_JPY_M5/2016-01-01_2022-07-31.csv
(491385, 26)
PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/GBP_JPY_H4/2016-01-01_2022-07-31.csv
(10666, 14)
GETTING SUPPORT INDICATOR
LOOKUP RANGE AND PIP DELTA: 2 0.5
COUNT: 0
TOTAL FUNCTION TIME: 0.008768129348754882  MINUTES
SHAPE (100, 27)
MEDIAN:
next_30             4.60
next_hour          -1.75
next_two_hours      9.35
next_four_hours    -0.25
next_eight_hours    3.05
dtype: float64

MEAN:
next_30              1.429
next_hour            6.441
next_two_hours      10.478
next_four_hours     -8.727
next_eight_hours   -10.849
dtype: float64

SUM:
   next_30  next_hour  next_two_hours  next_four_hours  next_eight_hours
0    142.9      644.1          1047.8           -872.7           -1084.9
None
EUR_USD
PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/EUR_USD_M5/2016-01-01_2022-07-31.csv
(490023, 26)
PATH: /Users/aidanmcconnell/Document

,next_30,next_hour,next_two_hours,next_four_hours,next_eight_hours
GBP_JPY,4.60,0.00,9.35,0.00,3.05
EUR_USD,2.80,3.15,5.35,0.00,3.85
USD_JPY,3.20,2.00,3.60,2.80,2.60
EUR_JPY,1.30,3.00,6.50,14.40,8.75
AUD_JPY,5.20,2.70,6.90,8.20,8.40
CAD_JPY,2.20,3.00,4.55,5.60,5.75
CHF_JPY,4.25,4.85,6.90,4.65,9.80
GBP_USD,2.40,5.20,4.55,2.95,10.20
USD_CAD,0.00,0.80,5.00,6.00,4.00
USD_CHF,0.55,0.45,1.70,1.90,9.40


In [304]:
def add_hour(df):
    df['hour'] = 0
    df['month'] = 0
    df['year'] = 0
    col1 = df.columns.get_loc('utc_timestamp')
    col2 = df.columns.get_loc('hour')
    col3 = df.columns.get_loc('month')
    col4 = df.columns.get_loc('year')
    
    arr = df.values 
    for i in range(arr.shape[0]):
        arr[i,col2] = pd.Timestamp(arr[i,col1]).hour
        arr[i,col3] = pd.Timestamp(arr[i,col1]).month
        arr[i,col4] = pd.Timestamp(arr[i,col1]).year
    return pd.DataFrame(arr,columns = df.columns)
df2 = load_df(pair = pair,granularity ='H4',start = datetime(2016,1,1,0,0,0),end = datetime(2022,7,31,0,0,0))
df2 = add_hour(df2)
df2.head()

PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/NZD_CHF_H4/2016-01-01_2022-07-31.csv
(10672, 14)


,time,volume,o,h,l,c,utc_timestamp,est_timestamp,delta_vector,delta_max,upper_wick,lower_wick,delta,minute_col,hour,month,year
0,2016-01-03T22:00:00.000000000Z,7404,0.68374,0.68374,0.67678,0.67683,2016-01-03 22:00:00+00:00,2016-01-03 17:00:00-05:00,-0.00691,0.00696,0.0,0.00005,0.00691,0,22,1,2016
1,2016-01-04T02:00:00.000000000Z,5726,0.67688,0.677,0.67132,0.67168,2016-01-04 02:00:00+00:00,2016-01-03 21:00:00-05:00,-0.0052,0.00568,0.00012,0.00036,0.0052,0,2,1,2016
2,2016-01-04T06:00:00.000000000Z,8797,0.67166,0.6739,0.67166,0.67246,2016-01-04 06:00:00+00:00,2016-01-04 01:00:00-05:00,0.0008,0.00224,0.00144,0.0,0.0008,0,6,1,2016
3,2016-01-04T10:00:00.000000000Z,7013,0.67246,0.67707,0.67208,0.67578,2016-01-04 10:00:00+00:00,2016-01-04 05:00:00-05:00,0.00332,0.00499,0.00129,0.00038,0.00332,0,10,1,2016
4,2016-01-04T14:00:00.000000000Z,9421,0.6758,0.67858,0.67514,0.67678,2016-01-04 14:00:00+00:00,2016-01-04 09:00:00-05:00,0.00098,0.00344,0.0018,0.00066,0.00098,0,14,1,2016


In [305]:
df2[df2['year'] == 2018].groupby('hour').agg({'month':max})

,month
hour,
1,11
2,12
5,11
6,12
9,11
10,12
13,11
14,12
17,11


In [306]:
df2[df2['year'] == 2018].groupby('hour').agg({'month':min})

,month
hour,
1,3
2,1
5,3
6,1
9,3
10,1
13,3
14,1
17,3


In [307]:
930 * 4

3720

In [296]:
pd.Timestamp(df2['utc_timestamp'].iloc[0]).day

3

In [308]:
120000000000 / 166000000

722.8915662650602

In [309]:
5000000 * 10000000

50000000000000

In [291]:
type(df2['utc_timestamp'].iloc[0])

str

In [ ]:
181

In [287]:
final_med.mean()

next_30             2.202381
next_hour           2.173810
next_two_hours      3.411905
next_four_hours     3.035714
next_eight_hours    4.388095
dtype: float64

In [283]:
final_sum

,next_30,next_hour,next_two_hours,next_four_hours,next_eight_hours
GBP_JPY,142.9,644.1,1047.8,0.0,0.0
EUR_USD,299.0,180.9,114.5,0.0,496.5
USD_JPY,75.4,92.8,380.2,0.0,0.0
EUR_JPY,352.6,707.9,911.4,754.1,1021.9
AUD_JPY,435.6,536.5,589.7,281.4,573.9
CAD_JPY,107.5,453.5,314.4,0.0,348.4
CHF_JPY,614.8,728.9,952.1,196.2,443.3
GBP_USD,233.6,472.6,563.4,0.0,121.9
USD_CAD,184.7,127.7,259.4,456.8,820.9
USD_CHF,6.9,0.0,85.4,258.6,763.0


In [284]:
final_sum.sum()

next_30             4818.7
next_hour           6014.4
next_two_hours      7137.2
next_four_hours     4240.9
next_eight_hours    6881.5
dtype: float64

In [281]:
final_med

,next_30,next_hour,next_two_hours,next_four_hours,next_eight_hours
GBP_JPY,2.00,0.00,3.60,0.00,0.00
EUR_USD,2.50,3.30,4.30,0.00,5.30
USD_JPY,3.20,5.60,3.00,2.80,1.10
EUR_JPY,1.00,4.20,5.00,8.60,2.60
AUD_JPY,4.00,2.30,2.70,5.10,0.20
CAD_JPY,2.60,4.15,2.85,2.00,6.60
CHF_JPY,5.40,5.80,5.90,3.10,5.80
GBP_USD,2.90,6.60,3.90,2.00,13.70
USD_CAD,0.00,0.35,3.40,3.30,1.05
USD_CHF,0.20,0.70,1.60,2.60,11.00


In [279]:
final_med

,next_30,next_hour,next_two_hours,next_four_hours,next_eight_hours
GBP_JPY,2.40,0.00,0.00,0.00,0.00
EUR_USD,1.50,1.80,3.90,1.20,2.40
USD_JPY,3.25,4.50,3.70,3.35,0.00
EUR_JPY,2.80,3.70,4.40,6.30,8.00
AUD_JPY,0.50,3.00,2.40,0.00,0.15
CAD_JPY,1.30,3.40,4.50,3.05,4.45
CHF_JPY,3.20,2.30,3.40,0.70,6.20
GBP_USD,1.75,2.60,1.65,3.25,1.25
USD_CAD,0.10,0.00,0.00,2.75,1.20
USD_CHF,0.40,0.00,1.30,1.55,4.65


In [276]:
final_sum

,next_30,next_hour,next_two_hours,next_four_hours,next_eight_hours
GBP_JPY,0.0,0.0,0.0,0.0,0.0
EUR_USD,119.7,34.4,265.1,433.5,996.8
USD_JPY,6.0,132.8,13.3,0.0,0.0
EUR_JPY,383.4,695.8,747.8,0.0,96.0
AUD_JPY,92.6,272.7,291.7,0.0,0.0
CAD_JPY,181.1,484.3,678.0,192.7,698.1
CHF_JPY,629.9,680.8,701.0,0.0,234.1
GBP_USD,232.3,36.7,248.7,93.6,0.0
USD_CAD,0.0,13.1,306.8,522.3,282.9
USD_CHF,0.0,0.0,307.6,283.6,939.9


In [277]:
final_mean

,next_30,next_hour,next_two_hours,next_four_hours,next_eight_hours
GBP_JPY,0.000000,0.000000,0.000000,0.000000,0.000000
EUR_USD,0.684000,0.196571,1.514857,2.477143,5.696000
USD_JPY,0.042857,0.948571,0.095000,0.000000,0.000000
EUR_JPY,2.442038,4.431847,4.763057,0.000000,0.611465
AUD_JPY,0.497849,1.466129,1.568280,0.000000,0.000000
CAD_JPY,0.953158,2.548947,3.568421,1.014211,3.674211
CHF_JPY,4.012102,4.336306,4.464968,0.000000,1.491083
GBP_USD,1.335057,0.210920,1.429310,0.537931,0.000000
USD_CAD,0.000000,0.077059,1.804706,3.072353,1.664118
USD_CHF,0.000000,0.000000,1.671739,1.541304,5.108152


In [228]:
for pair in all_pairs:
    print(pair)  

    if 'JPY' in pair:
        sls = [.05,.075,.1,.125,.15,.2,.25]
        tps = [.1,.15,.2,.25,.3,.4,.5,.7,.9,1.1,1.3,1.5]
        spread = .01
        high_spread = .0005 * 100
        low_spread = 0 * 100
    else:
        sls = [.0005,.00075,.001,.00125,.0015,.002,.0025]
        tps = [.001,.0015,.002,.0025,.003,.004,.005,.007,.009,.011,.013,.015]
        spread = .0001
        high_spread = .0005
        low_spread = 0

    df = load_df(pair = pair,granularity ='M5',start = datetime(2016,1,1,0,0,0),end = datetime(2022,7,31,0,0,0))
    df = get_support(df,
                lookup_range = 2,
                pip_delta = .002,
                 time_range = 47,
                high_spread = high_spread,
                low_spread = low_spread
               )
    print(df.shape)
    for sl in sls:
        for tp in tps:

            val,win_rate,wins,losses = strategy_tester_sell(df,col = 'support_indicator',sl = sl,tp = tp,num_trades = 400,risk_size = .01,entry = 'c',pr = False,spread = spread)
            print(sl,tp,val)

GBP_JPY
PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/GBP_JPY_M5/2016-01-01_2022-07-31.csv
(491385, 26)
GETTING SUPPORT INDICATOR
LOOKUP RANGE AND PIP DELTA: 2 0.002
COUNT: 0
TOTAL FUNCTION TIME: 0.023795819282531737  MINUTES
SHAPE (27, 27)
(491385, 27)
0.05 0.1 1591.5
0.05 0.15 2224.0
0.05 0.2 2130.0
0.05 0.25 1608.5
0.05 0.3 2167.0
0.05 0.4 5163.0
0.05 0.5 2241.5
0.05 0.7 842.0
0.05 0.9 1662.0
0.05 1.1 1965.5
0.05 1.3 3515.5
0.05 1.5 4611.0
0.075 0.1 3661.5
0.075 0.15 1854.5
0.075 0.2 2002.0
0.075 0.25 1556.0
0.075 0.3 2155.0
0.075 0.4 2184.0
0.075 0.5 1304.0
0.075 0.7 2532.5
0.075 0.9 983.5
0.075 1.1 1050.5
0.075 1.3 1738.0
0.075 1.5 3055.5
0.1 0.1 3486.0
0.1 0.15 1859.0
0.1 0.2 2016.0
0.1 0.25 1895.0
0.1 0.3 2592.5
0.1 0.4 1353.0
0.1 0.5 850.0
0.1 0.7 1402.0
0.1 0.9 614.0
0.1 1.1 776.0
0.1 1.3 865.0
0.1 1.5 1727.5
0.125 0.1 4354.0
0.125 0.15 2535.5
0.125 0.2 4437.5
0.125 0.25 6644.0
0.125 0.3 7449.5
0.125 0.4 5329.0
0.125 0.5 5047.0
0.125 0.7 17025.0
0.125 0.9 

In [226]:
def strategy_tester_buy(df,col = 'support_indicator',sl = .1,tp = .5,num_trades = 300,risk_size = .01,entry = 'c',pr = True,spread = .01):
    def simulate_results(
        num_trades = 200,
        acc_val = 10000,
        rr = 6,
        risk_size = .02,
        win_rate = .2,
        pr = True):

        start = acc_val
        for i in range(num_trades):
            rand = random.random()
            if rand > win_rate:
                acc_val = acc_val - (acc_val * risk_size)
            else:
                acc_val = acc_val + (acc_val * (risk_size * rr))
        if pr:
            print('STARTING VAL:',round(start),' ENDING VAL:',round(acc_val))
        return round(acc_val)

    col_num = df.columns.get_loc(col)
    h_col = df.columns.get_loc('h')
    l_col = df.columns.get_loc('l')
    c_col = df.columns.get_loc('c')
    entry_col = df.columns.get_loc(entry)
    arr = df.values
    temp_df = df[df[col] == 1]

    trade_list = []
    trade_res = []
    wins = []
    losses = []
    for i in range(arr.shape[0]):

        if arr[i,col_num] == 1:
            val = arr[i,entry_col]
            trade_id = 0
            try:
                for j in range(i + 1,i + 10000):
                    if i not in trade_list:
                        if trade_id == 0:
                            if arr[j,l_col] <= val - sl + spread:
                                trade_res.append(-10)
                                trade_list.append(i)
                                losses.append(i)
                                trade_id = 1

                                break
                            if arr[j,h_col] >= val + tp:
                                trade_res.append(50)
                                trade_list.append(i)
                                wins.append(i)
                                trade_id = 1
                                break
            except:
                pass
    win_rate = len(wins) / len(trade_list)
    rr = (tp - spread) / (sl + spread)
   # num_trades = len(trade_list)
  #  print('WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size)
    vals = []


    acc_val = 10000



    for i in range(30):

        final_val = simulate_results(num_trades = num_trades,acc_val = acc_val,rr = rr,risk_size = risk_size,win_rate = win_rate,pr = pr)
        vals.append(final_val)

    return np.median(vals),win_rate,wins,losses

def strategy_tester_sell(df,col = 'support_indicator',sl = .1,tp = .5,num_trades = 300,risk_size = .01,entry = 'c',pr = True,spread = .01):
    def simulate_results(
        num_trades = 200,
        acc_val = 10000,
        rr = 6,
        risk_size = .02,
        win_rate = .2,
        pr = True):

        start = acc_val
        for i in range(num_trades):
            rand = random.random()
            if rand > win_rate:
                acc_val = acc_val - (acc_val * risk_size)
            else:
                acc_val = acc_val + (acc_val * (risk_size * rr))
        if pr:
            print('STARTING VAL:',round(start),' ENDING VAL:',round(acc_val))
        return round(acc_val)

    col_num = df.columns.get_loc(col)
    h_col = df.columns.get_loc('h')
    l_col = df.columns.get_loc('l')
    c_col = df.columns.get_loc('c')
    entry_col = df.columns.get_loc(entry)
    arr = df.values
    temp_df = df[df[col] == 1]

    trade_list = []
    trade_res = []
    wins = []
    losses = []
    for i in range(arr.shape[0]):

        if arr[i,col_num] == 1:
            val = arr[i,entry_col]
            try:
                for j in range(i + 1,i + 10000):
                    if i not in trade_list:
                        if arr[j,h_col] >= val + sl - spread:
                            trade_res.append(-10)
                            trade_list.append(i)
                            losses.append(i)
                            break
                        if arr[j,l_col] <= val - tp:
                            trade_res.append(50)
                            trade_list.append(i)
                            wins.append(i)
                            break
            except:
                pass
    win_rate = len(wins) / len(trade_list)
    rr = (tp - spread) / (sl + spread)
   # num_trades = len(trade_list)
  #  print('WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size)
    vals = []


    acc_val = 10000



    for i in range(30):

        final_val = simulate_results(num_trades = num_trades,acc_val = acc_val,rr = rr,risk_size = risk_size,win_rate = win_rate,pr = pr)
        vals.append(final_val)

    return np.median(vals),win_rate,wins,losses


sls = [.0005,.00075,.001,.00125,.0015,.002,.0025]
tps = [.001,.0015,.002,.0025,.003,.004,.005,.007,.009,.011,.013,.015]
spread = .0001
for sl in sls:
    for tp in tps:

        val,win_rate,wins,losses = strategy_tester_buy(df,col = 'support_indicator',sl = sl,tp = tp,num_trades = 400,risk_size = .01,entry = 'c',pr = False,spread = spread)
        print(sl,tp,val)



0.0005 0.001 30189.5
0.0005 0.0015 47587.5
0.0005 0.002 98497.0
0.0005 0.0025 165140.5
0.0005 0.003 575347.0
0.0005 0.004 1029260.5
0.0005 0.005 497183.5
0.0005 0.007 86964.0
0.0005 0.009 105849.0
0.0005 0.011 67698.5
0.0005 0.013 189703.0
0.0005 0.015 377594.0
0.00075 0.001 31468.0
0.00075 0.0015 31223.0
0.00075 0.002 66678.5
0.00075 0.0025 75700.0
0.00075 0.003 233856.0
0.00075 0.004 391514.0
0.00075 0.005 516816.5
0.00075 0.007 132948.0
0.00075 0.009 75043.5
0.00075 0.011 43470.0
0.00075 0.013 120339.0
0.00075 0.015 340862.0
0.001 0.001 16982.0
0.001 0.0015 17195.0
0.001 0.002 34033.0
0.001 0.0025 38870.0
0.001 0.003 61941.0
0.001 0.004 144198.0
0.001 0.005 207563.5
0.001 0.007 88752.5
0.001 0.009 49843.5
0.001 0.011 50768.0
0.001 0.013 131048.0
0.001 0.015 223808.0
0.00125 0.001 14366.0
0.00125 0.0015 14028.0
0.00125 0.002 25026.5
0.00125 0.0025 27626.0
0.00125 0.003 58777.0
0.00125 0.004 133020.0
0.00125 0.005 174352.5
0.00125 0.007 59894.0
0.00125 0.009 49459.0
0.00125 0.011 5141

In [224]:

def viz_support(df,col,start,end,ema):
    print('SELL INTO SUPPORT!!!!!')
    def print_support_example(df,col,ind_list,ind = 30,delta = 240,ema = True):
        i = ind_list[ind]   
        x = i


        print('SUPPORT ROW:',x,' SUPPORT NEXT TOUCH:',i, ' SUPPORT CLOSE VAL:')

        fig = go.Figure(data=go.Candlestick(x=df.iloc[i - delta:i + delta,:]['time'],
                            open=df.iloc[x - delta:i + delta,:]['o'],
                            high=df.iloc[x - delta:i + delta,:]['h'],
                            low=df.iloc[x - delta:i + delta,:]['l'],
                            close=df.iloc[x - delta:i + delta,:]['c']))   


        fig.add_vrect(x0=df['time'].iloc[i - 1], x1=df['time'].iloc[i+1], 
                      annotation_text="trigger point" , annotation_position="top left",
                      fillcolor="green", opacity=0.25, line_width=0)

        fig.add_shape(type='line',
                        x0=df.iloc[x,:]['time'],
                        y0=df.iloc[x,:]['c'],
                        x1=df.iloc[i,:]['time'],
                        y1=df.iloc[x ,:]['c'],
                        line=dict(color='Red',),
                        xref='x',
                        yref='y'
        )
        if ema:

            fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['ema_42'],line=dict(color='Yellow',)))
            fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['ema_150'],line=dict(color='Blue',)))
            fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['ema_600'],line=dict(color='Black',)))
        else:
            fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['smma_21'],line=dict(color='Yellow',)))
            fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['smma_50'],line=dict(color='Blue',)))
            fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['smma_200'],line=dict(color='Black',)))
        try:
            plot(fig)
        except:
            from plotly.offline import plot
            plot(fig)
            
            
    temp_df = df[df[col] == 1]
    print('SHAPE:',temp_df.shape)
    ind_list = temp_df.index
    for i in range(start,end):
        print(i)

        #i = 3
        print_support_example(df,col,ind_list,ind = i,delta = 100,ema = ema)
        i += 1

        
viz_support(df = df,col = 'support_indicator',start = 5,end = 15,ema = True)



SELL INTO SUPPORT!!!!!
SHAPE: (37, 27)
5
SUPPORT ROW: 60416  SUPPORT NEXT TOUCH: 60416  SUPPORT CLOSE VAL:
6
SUPPORT ROW: 63358  SUPPORT NEXT TOUCH: 63358  SUPPORT CLOSE VAL:
7
SUPPORT ROW: 66610  SUPPORT NEXT TOUCH: 66610  SUPPORT CLOSE VAL:
8
SUPPORT ROW: 67518  SUPPORT NEXT TOUCH: 67518  SUPPORT CLOSE VAL:
9
SUPPORT ROW: 68873  SUPPORT NEXT TOUCH: 68873  SUPPORT CLOSE VAL:
10
SUPPORT ROW: 82010  SUPPORT NEXT TOUCH: 82010  SUPPORT CLOSE VAL:
11
SUPPORT ROW: 133218  SUPPORT NEXT TOUCH: 133218  SUPPORT CLOSE VAL:
12
SUPPORT ROW: 137838  SUPPORT NEXT TOUCH: 137838  SUPPORT CLOSE VAL:
13
SUPPORT ROW: 147658  SUPPORT NEXT TOUCH: 147658  SUPPORT CLOSE VAL:
14
SUPPORT ROW: 152092  SUPPORT NEXT TOUCH: 152092  SUPPORT CLOSE VAL:


In [189]:
max(2,5)

5

In [ ]:
def get_support(df,
                lookup_range = 1500,
                stop_range = 2200,
                bound_pips = .001,
                bound_pips2 = .005
               ):
    print('GETTING SUPPORT INDICATOR')
    s = time.time()
    c_col = df.columns.get_loc('c')
    o_col = df.columns.get_loc('o')
    l_col = df.columns.get_loc('l')
    h_col = df.columns.get_loc('h')
    d_col = df.columns.get_loc('delta')
    df['support_lookup'] = 0
    df['support_indicator'] = 0

    lookup_col = df.columns.get_loc('support_lookup')
    new_col = df.columns.get_loc('support_indicator')

    reversal_col = df.columns.get_loc('c')   

    arr = df.values
    min_count = 0

    print('STOP RANGE:',stop_range)
    c = 0 

    for i in range(lookup_range,arr.shape[0]):
       # try:
        #DEFINE SUPPORT/RESISTANCE as the MIN or MAX of a lookup range
        #min_ = min(arr[i - lookup_range : i + lookup_range2,reversal_col])
        min_ = min(arr[i - lookup_range : i + 1,reversal_col])

        
        reversal_ind = 0
        if arr[i,reversal_col] == min_:
            for k in range(i,i + stop_range):
                if k >= arr.shape[0]:
                    ind = 0
                    break
                if arr[k,reversal_col] < min_:
                    break
                if arr[k,reversal_col] - min_ >= bound_pips2 :
                    reversal_ind = 1
                    ind = k
                    break
                
        # If the current val is at that reversal point:
        # Check and make sure we have an adequate movement off of that reversal point
        # this is defined by the pip_range and pip threshold, pip threshold is a user defined parameter
        if reversal_ind == 1 and ind != 0:
            lookup_ind = i
            val = arr[i,reversal_col]
            lookup_check = 0
            # Now that all of these conditions are met
            # We need to look into the future to see if we hit this reversal level, this is defined by the stop range
            if ind + stop_range < df.shape[0]:
                for j in range(ind,ind + stop_range):

                    lower_bound = val - bound_pips 
                    upper_bound = val + bound_pips                    
                    cur_val = arr[j, reversal_col]
                    max_val = max(arr[j:j + 1000, reversal_col])
                    second_delta = max_val - val
                

                    if cur_val < lower_bound and arr[j + 24,reversal_col] < lower_bound:
                        lookup_check = 1

                    if lookup_check == 0 and \
                    cur_val <= upper_bound and \
                    cur_val >= lower_bound and \
                    second_delta >= bound_pips2 and \
                    arr[j + 30,reversal_col] > val:
                        if j + 30 + (stop_range * 3) >= df.shape[0]:
                            loop_val = df.shape[0]
                        else:
                            loop_val = j + 30 + (stop_range * 3)
                        for k in range(j + 30, loop_val):
                            cur_val2 = arr[k, reversal_col]
                            
                            if lookup_check == 0 and \
                            cur_val2 <= upper_bound and \
                            cur_val2 >= lower_bound and \
                            max(arr[j:k,reversal_col]) - val >= bound_pips2:                     
                                arr[k,new_col] = 1
                                arr[k,lookup_col] = lookup_ind
                                lookup_check = 1
                                break

    e = time.time()
    print('TOTAL FUNCTION TIME:',(e-s)/60,' MINUTES')
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['support_indicator'] == 1].shape)
    return df



pair = 'EUR_USD'
df = load_df(pair = pair,granularity ='M5',start = datetime(2016,1,1,0,0,0),end = datetime(2022,7,31,0,0,0))
print(df.shape)
df = get_support(df,
                lookup_range = 300,
                stop_range = 10000,
                bound_pips = .001,
                bound_pips2 = .005
               )
print(df.shape)
df.head()


In [64]:
df.columns

Index(['time', 'volume', 'o', 'h', 'l', 'c', 'utc_timestamp', 'est_timestamp',
       'smma_21', 'smma_50', 'smma_200', 'ema_42', 'ema_150', 'ema_600',
       'bearish_engulfing', 'bullish_engulfing', 'bearish_tls', 'bullish_tls',
       'new_york', 'london', 'delta_vector', 'delta_max', 'upper_wick',
       'lower_wick', 'delta', 'support_lookup', 'support_indicator'],
      dtype='object')

In [44]:
400 * 5 / 60

33.333333333333336

In [191]:
def viz_support(df,start,end):
    print('SELL INTO SUPPORT!!!!!')
    def print_support_example(df,ind_list,ind = 30,delta = 240,ema = True):
        i = ind_list[ind]   
        x = df['support_lookup'].iloc[i]


        print('SUPPORT ROW:',x,' SUPPORT NEXT TOUCH:',i, ' SUPPORT CLOSE VAL:',df['c'].iloc[x])

        fig = go.Figure(data=go.Candlestick(x=df.iloc[x - delta:i + delta,:]['time'],
                            open=df.iloc[x - delta:i + delta,:]['o'],
                            high=df.iloc[x - delta:i + delta,:]['h'],
                            low=df.iloc[x - delta:i + delta,:]['l'],
                            close=df.iloc[x - delta:i + delta,:]['c']))   






        fig.add_vrect(x0=df['time'].iloc[i - 1], x1=df['time'].iloc[i+1], 
                      annotation_text="trigger point" , annotation_position="top left",
                      fillcolor="green", opacity=0.25, line_width=0)

        fig.add_shape(type='line',
                        x0=df.iloc[x,:]['time'],
                        y0=df.iloc[x,:]['c'],
                        x1=df.iloc[i,:]['time'],
                        y1=df.iloc[x ,:]['c'],
                        line=dict(color='Red',),
                        xref='x',
                        yref='y'
        )
        
        if ema:

            fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['ema_210'],line=dict(color='Yellow',)))
            fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['ema_750'],line=dict(color='Blue',)))
            fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['ema_3000'],line=dict(color='Black',)))
        else:
            fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['smma_21'],line=dict(color='Yellow',)))
            fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['smma_50'],line=dict(color='Blue',)))
            fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['smma_200'],line=dict(color='Black',)))
        try:
            plot(fig)
        except:
            from plotly.offline import plot
            plot(fig)
            
            
    temp_df = df[df['support_indicator'] == 1]
    print('SHAPE:',temp_df.shape)
    ind_list = temp_df.index
    for i in range(start,end):
        print(i)

        #i = 3
        print_support_example(df,ind_list,ind = i,delta = 1000)
        i += 1
        

def viz_support(df,col,start,end,ema):
    print('SELL INTO SUPPORT!!!!!')
    def print_support_example(df,col,ind_list,ind = 30,delta = 240,ema = True):
        i = ind_list[ind]   
        x = df['support_lookup'].iloc[i]


        print('SUPPORT ROW:',x,' SUPPORT NEXT TOUCH:',i, ' SUPPORT CLOSE VAL:',df['c'].iloc[x])

        fig = go.Figure(data=go.Candlestick(x=df.iloc[x - delta:i + delta,:]['time'],
                            open=df.iloc[x - delta:i + delta,:]['o'],
                            high=df.iloc[x - delta:i + delta,:]['h'],
                            low=df.iloc[x - delta:i + delta,:]['l'],
                            close=df.iloc[x - delta:i + delta,:]['c']))   


        fig.add_vrect(x0=df['time'].iloc[i - 1], x1=df['time'].iloc[i+1], 
                      annotation_text="trigger point" , annotation_position="top left",
                      fillcolor="green", opacity=0.25, line_width=0)

        fig.add_shape(type='line',
                        x0=df.iloc[x,:]['time'],
                        y0=df.iloc[x,:]['c'],
                        x1=df.iloc[i,:]['time'],
                        y1=df.iloc[x ,:]['c'],
                        line=dict(color='Red',),
                        xref='x',
                        yref='y'
        )
        if ema:

            fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['ema_42'],line=dict(color='Yellow',)))
            fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['ema_150'],line=dict(color='Blue',)))
            fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['ema_600'],line=dict(color='Black',)))
        else:
            fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['smma_21'],line=dict(color='Yellow',)))
            fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['smma_50'],line=dict(color='Blue',)))
            fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['smma_200'],line=dict(color='Black',)))
        try:
            plot(fig)
        except:
            from plotly.offline import plot
            plot(fig)
            
            
    temp_df = df[df[col] == 1]
    print('SHAPE:',temp_df.shape)
    ind_list = temp_df.index
    for i in range(start,end):
        print(i)

        #i = 3
        print_support_example(df,col,ind_list,ind = i,delta = 1000,ema = ema)
        i += 1
        
        
viz_support(df = df,col = 'support_indicator',start = 20,end = 30,ema = True)


SELL INTO SUPPORT!!!!!
SHAPE: (68, 27)
20


KeyError: 'support_lookup'

In [48]:
df.columns

Index(['time', 'volume', 'o', 'h', 'l', 'c', 'utc_timestamp', 'est_timestamp',
       'smma_21', 'smma_50', 'smma_200', 'ema_42', 'ema_150', 'ema_600',
       'bearish_engulfing', 'bullish_engulfing', 'bearish_tls', 'bullish_tls',
       'new_york', 'london', 'delta_vector', 'delta_max', 'upper_wick',
       'lower_wick', 'delta', 'support_lookup', 'support_indicator'],
      dtype='object')

In [73]:
def strategy_tester_buy(df,col = 'support_indicator',sl = .1,tp = .5,num_trades = 300,risk_size = .01,entry = 'c',pr = True,spread = .01):
    def simulate_results(
        num_trades = 200,
        acc_val = 10000,
        rr = 6,
        risk_size = .02,
        win_rate = .2,
        pr = True):

        start = acc_val
        for i in range(num_trades):
            rand = random.random()
            if rand > win_rate:
                acc_val = acc_val - (acc_val * risk_size)
            else:
                acc_val = acc_val + (acc_val * (risk_size * rr))
        if pr:
            print('STARTING VAL:',round(start),' ENDING VAL:',round(acc_val))
        return round(acc_val)

    col_num = df.columns.get_loc(col)
    h_col = df.columns.get_loc('h')
    l_col = df.columns.get_loc('l')
    c_col = df.columns.get_loc('c')
    entry_col = df.columns.get_loc(entry)
    arr = df.values
    temp_df = df[df[col] == 1]

    trade_list = []
    trade_res = []
    wins = []
    losses = []
    for i in range(arr.shape[0]):

        if arr[i,col_num] == 1:
            val = arr[i,entry_col]
            trade_id = 0
            try:
                for j in range(i + 1,i + 10000):
                    if i not in trade_list:
                        if trade_id == 0:
                            if arr[j,l_col] <= val - sl + spread:
                                trade_res.append(-10)
                                trade_list.append(i)
                                losses.append(i)
                                trade_id = 1

                                break
                            if arr[j,h_col] >= val + tp:
                                trade_res.append(50)
                                trade_list.append(i)
                                wins.append(i)
                                trade_id = 1
                                break
            except:
                pass
    win_rate = len(wins) / len(trade_list)
    rr = (tp - spread) / (sl + spread)
   # num_trades = len(trade_list)
  #  print('WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size)
    vals = []


    acc_val = 10000



    for i in range(30):

        final_val = simulate_results(num_trades = num_trades,acc_val = acc_val,rr = rr,risk_size = risk_size,win_rate = win_rate,pr = pr)
        vals.append(final_val)

    return np.median(vals),win_rate,wins,losses

def strategy_tester_sell(df,col = 'support_indicator',sl = .1,tp = .5,num_trades = 300,risk_size = .01,entry = 'c',pr = True,spread = .01):
    def simulate_results(
        num_trades = 200,
        acc_val = 10000,
        rr = 6,
        risk_size = .02,
        win_rate = .2,
        pr = True):

        start = acc_val
        for i in range(num_trades):
            rand = random.random()
            if rand > win_rate:
                acc_val = acc_val - (acc_val * risk_size)
            else:
                acc_val = acc_val + (acc_val * (risk_size * rr))
        if pr:
            print('STARTING VAL:',round(start),' ENDING VAL:',round(acc_val))
        return round(acc_val)

    col_num = df.columns.get_loc(col)
    h_col = df.columns.get_loc('h')
    l_col = df.columns.get_loc('l')
    c_col = df.columns.get_loc('c')
    entry_col = df.columns.get_loc(entry)
    arr = df.values
    temp_df = df[df[col] == 1]

    trade_list = []
    trade_res = []
    wins = []
    losses = []
    for i in range(arr.shape[0]):

        if arr[i,col_num] == 1:
            val = arr[i,entry_col]
            try:
                for j in range(i + 1,i + 10000):
                    if i not in trade_list:
                        if arr[j,h_col] >= val + sl - spread:
                            trade_res.append(-10)
                            trade_list.append(i)
                            losses.append(i)
                            break
                        if arr[j,l_col] <= val - tp:
                            trade_res.append(50)
                            trade_list.append(i)
                            wins.append(i)
                            break
            except:
                pass
    win_rate = len(wins) / len(trade_list)
    rr = (tp - spread) / (sl + spread)
   # num_trades = len(trade_list)
  #  print('WIN RATE',win_rate,' RR:',rr, 'NUM TRADES:',num_trades,' RISK SIZE:',risk_size)
    vals = []


    acc_val = 10000



    for i in range(30):

        final_val = simulate_results(num_trades = num_trades,acc_val = acc_val,rr = rr,risk_size = risk_size,win_rate = win_rate,pr = pr)
        vals.append(final_val)

    return np.median(vals),win_rate,wins,losses




cols = ['instrument',
           'buy_support_val',
           'sell_support_val',
            'max_ending_val',
            'sl',
            'tp',
            'RR',
           'win_rate1',
           'win_rate4',
            'support_number_of_trades'
          ]

trade_strategy = 'support_indicator'


#trade_strategy = 'resistance_indicatorc'
num_trades = 400
risk_size = .01

starting_val = 10000
instrument = pair
timeframe = 'M5'
temp_lst = []
actual_number_of_trades = df[df[trade_strategy] == 1].shape[0]
sls = [.0005,.00075,.001,.00125,.0015,.002,.0025]
tps = [.001,.0015,.002,.0025,.003,.004,.005,.007,.009,.011,.013,.015]
spread = .0001      

for sl in sls:
    for tp in tps:
        if actual_number_of_trades == 0:
            ending_val,win_rate,wins,losses = 0,0,0,0
            ending_val4,win_rate4,wins4,losses4 = 0,0,0,0
        else:
            ending_val,win_rate,wins,losses = strategy_tester_buy(df,col = trade_strategy,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False,spread = spread)
            ending_val4,win_rate4,wins4,losses4 = strategy_tester_sell(df,col = trade_strategy,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False,spread = spread)   



        max_ending_val = max(ending_val,
               ending_val4)
        lst = [instrument,
               ending_val,
               ending_val4,
               max_ending_val,
                sl,
                tp,
                round(tp / sl,4),
               round(win_rate,4),
               round(win_rate4,4),
               actual_number_of_trades
              ]
        temp_lst.append(lst)

final_df = pd.DataFrame(temp_lst,columns = cols)
final_df = final_df.sort_values(by = 'max_ending_val',ascending = False)

e = time.time()
best_buy_support = final_df['buy_support_val'].max()
best_sell_support = final_df['sell_support_val'].max()
max_val_all = final_df['max_ending_val'].max()
print('MAX VAL:',round(max_val_all),'||best buy support:',round(best_buy_support),'||best sell support:',round(best_sell_support))




MAX VAL: 23230 ||best buy support: 23074 ||best sell support: 23230


In [88]:
def support_indicator_2(df,candle_size = .001):
    df['support_indicator2'] = 0
    new_col = df.columns.get_loc('support_indicator2')
    col = df.columns.get_loc('support_indicator')
    lookup_col = df.columns.get_loc('support_lookup')
    c_col = df.columns.get_loc('c')
    o_col = df.columns.get_loc('o')
    l_col = df.columns.get_loc('l')
    arr = df.values
    
    for i in range(arr.shape[0]):
        lookup_check = 0
        if arr[i,col] == 1:
            for j in range(i - 1,i + 40):
                if arr[j,c_col] < arr[i,c_col] and arr[j,o_col] - arr[j,c_col] >= candle_size and lookup_check == 0 and arr[j,c_col] - arr[j,l_col] <= .00015:
                    arr[j,new_col] = 1
                    arr[j,lookup_col] = arr[i,lookup_col]
                    lookup_check = 1
    df = pd.DataFrame(arr,columns = df.columns)
    print('NEW SHAPE',df[df['support_indicator2'] == 1].shape[0])
    
    return df
df = support_indicator_2(df,candle_size = .0008)


cols = ['instrument',
           'buy_support_val',
           'sell_support_val',
            'max_ending_val',
            'sl',
            'tp',
            'RR',
           'win_rate1',
           'win_rate4',
            'support_number_of_trades'
          ]

trade_strategy = 'support_indicator2'


#trade_strategy = 'resistance_indicatorc'
num_trades = 400
risk_size = .01

starting_val = 10000
instrument = pair
timeframe = 'M5'
temp_lst = []
actual_number_of_trades = df[df[trade_strategy] == 1].shape[0]
sls = [.0005,.00075,.001,.00125,.0015,.002,.0025]
tps = [.001,.0015,.002,.0025,.003,.004,.005,.007,.009,.011,.013,.015]
spread = .0001      

for sl in sls:
    for tp in tps:
        if actual_number_of_trades == 0:
            ending_val,win_rate,wins,losses = 0,0,0,0
            ending_val4,win_rate4,wins4,losses4 = 0,0,0,0
        else:
            ending_val,win_rate,wins,losses = strategy_tester_buy(df,col = trade_strategy,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False,spread = spread)
            ending_val4,win_rate4,wins4,losses4 = strategy_tester_sell(df,col = trade_strategy,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False,spread = spread)   



        max_ending_val = max(ending_val,
               ending_val4)
        lst = [instrument,
               ending_val,
               ending_val4,
               max_ending_val,
                sl,
                tp,
                round(tp / sl,4),
               round(win_rate,4),
               round(win_rate4,4),
               actual_number_of_trades
              ]
        temp_lst.append(lst)

final_df = pd.DataFrame(temp_lst,columns = cols)
final_df = final_df.sort_values(by = 'max_ending_val',ascending = False)

e = time.time()
best_buy_support = final_df['buy_support_val'].max()
best_sell_support = final_df['sell_support_val'].max()
max_val_all = final_df['max_ending_val'].max()
print('MAX VAL:',round(max_val_all),'||best buy support:',round(best_buy_support),'||best sell support:',round(best_sell_support))
final_df.head()


NEW SHAPE 55
MAX VAL: 45124 ||best buy support: 45124 ||best sell support: 26783


,instrument,buy_support_val,sell_support_val,max_ending_val,sl,tp,RR,win_rate1,win_rate4,support_number_of_trades
22,EUR_USD,45124.5,22772.0,45124.5,0.00075,0.013,17.3333,0.0909,0.0727,55
46,EUR_USD,42661.0,16315.5,42661.0,0.00125,0.013,10.4000,0.1273,0.1091,55
11,EUR_USD,33315.0,4611.0,33315.0,0.00050,0.015,30.0000,0.0545,0.0364,55
70,EUR_USD,32239.0,6268.0,32239.0,0.00200,0.013,6.5000,0.1818,0.1273,55
58,EUR_USD,28976.5,15635.0,28976.5,0.00150,0.013,8.6667,0.1455,0.1273,55


In [91]:
def support_indicator_2(df,candle_size = .001):
    df['support_indicator2'] = 0
    new_col = df.columns.get_loc('support_indicator2')
    col = df.columns.get_loc('support_indicator')
    lookup_col = df.columns.get_loc('support_lookup')
    c_col = df.columns.get_loc('c')
    o_col = df.columns.get_loc('o')
    l_col = df.columns.get_loc('l')
    arr = df.values
    
    for i in range(arr.shape[0]):
        lookup_check = 0
        if arr[i,col] == 1:
            for j in range(i - 1,i + 40):
                if arr[j,c_col] < arr[i,c_col] and lookup_check == 0 and arr[j,c_col] - arr[j,l_col] >= .0008:
                    arr[j,new_col] = 1
                    arr[j,lookup_col] = arr[i,lookup_col]
                    lookup_check = 1
    df = pd.DataFrame(arr,columns = df.columns)
    print('NEW SHAPE',df[df['support_indicator2'] == 1].shape[0])
    
    return df
df = support_indicator_2(df,candle_size = .0008)


cols = ['instrument',
           'buy_support_val',
           'sell_support_val',
            'max_ending_val',
            'sl',
            'tp',
            'RR',
           'win_rate1',
           'win_rate4',
            'support_number_of_trades'
          ]

trade_strategy = 'support_indicator2'


#trade_strategy = 'resistance_indicatorc'
num_trades = 400
risk_size = .01

starting_val = 10000
instrument = pair
timeframe = 'M5'
temp_lst = []
actual_number_of_trades = df[df[trade_strategy] == 1].shape[0]
sls = [.0005,.00075,.001,.00125,.0015,.002,.0025]
tps = [.001,.0015,.002,.0025,.003,.004,.005,.007,.009,.011,.013,.015]
spread = .0001      

for sl in sls:
    for tp in tps:
        if actual_number_of_trades == 0:
            ending_val,win_rate,wins,losses = 0,0,0,0
            ending_val4,win_rate4,wins4,losses4 = 0,0,0,0
        else:
            ending_val,win_rate,wins,losses = strategy_tester_buy(df,col = trade_strategy,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False,spread = spread)
            ending_val4,win_rate4,wins4,losses4 = strategy_tester_sell(df,col = trade_strategy,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False,spread = spread)   



        max_ending_val = max(ending_val,
               ending_val4)
        lst = [instrument,
               ending_val,
               ending_val4,
               max_ending_val,
                sl,
                tp,
                round(tp / sl,4),
               round(win_rate,4),
               round(win_rate4,4),
               actual_number_of_trades
              ]
        temp_lst.append(lst)

final_df = pd.DataFrame(temp_lst,columns = cols)
final_df = final_df.sort_values(by = 'max_ending_val',ascending = False)

e = time.time()
best_buy_support = final_df['buy_support_val'].max()
best_sell_support = final_df['sell_support_val'].max()
max_val_all = final_df['max_ending_val'].max()
print('MAX VAL:',round(max_val_all),'||best buy support:',round(best_buy_support),'||best sell support:',round(best_sell_support))
final_df.head()

NEW SHAPE 71
MAX VAL: 85903 ||best buy support: 38235 ||best sell support: 85903


,instrument,buy_support_val,sell_support_val,max_ending_val,sl,tp,RR,win_rate1,win_rate4,support_number_of_trades
71,EUR_USD,20873.0,85903.0,85903.0,0.00200,0.015,7.5,0.1549,0.1972,71
59,EUR_USD,4969.0,79660.0,79660.0,0.00150,0.015,10.0,0.0845,0.1549,71
47,EUR_USD,2241.0,78561.0,78561.0,0.00125,0.015,12.0,0.0563,0.1268,71
35,EUR_USD,930.0,69960.0,69960.0,0.00100,0.015,15.0,0.0282,0.1127,71
69,EUR_USD,27618.0,64443.0,64443.0,0.00200,0.011,5.5,0.2113,0.2394,71


In [93]:
def support_indicator_2(df,candle_size = .001):
    df['support_indicator2'] = 0
    new_col = df.columns.get_loc('support_indicator2')
    col = df.columns.get_loc('support_indicator')
    lookup_col = df.columns.get_loc('support_lookup')
    c_col = df.columns.get_loc('c')
    o_col = df.columns.get_loc('o')
    l_col = df.columns.get_loc('l')
    h_col = df.columns.get_loc('h')
    arr = df.values
    
    for i in range(arr.shape[0]):
        lookup_check = 0
        if arr[i,col] == 1:
            for j in range(i - 1,i + 40):
                if arr[j,c_col] > arr[i,c_col] and lookup_check == 0 and arr[j,h_col] - arr[j,c_col] >= .0008:
                    arr[j,new_col] = 1
                    arr[j,lookup_col] = arr[i,lookup_col]
                    lookup_check = 1
    df = pd.DataFrame(arr,columns = df.columns)
    print('NEW SHAPE',df[df['support_indicator2'] == 1].shape[0])
    
    return df
df = support_indicator_2(df,candle_size = .0008)


cols = ['instrument',
           'buy_support_val',
           'sell_support_val',
            'max_ending_val',
            'sl',
            'tp',
            'RR',
           'win_rate1',
           'win_rate4',
            'support_number_of_trades'
          ]

trade_strategy = 'support_indicator2'


#trade_strategy = 'resistance_indicatorc'
num_trades = 400
risk_size = .01

starting_val = 10000
instrument = pair
timeframe = 'M5'
temp_lst = []
actual_number_of_trades = df[df[trade_strategy] == 1].shape[0]
sls = [.0005,.00075,.001,.00125,.0015,.002,.0025]
tps = [.001,.0015,.002,.0025,.003,.004,.005,.007,.009,.011,.013,.015]
spread = .0001      

for sl in sls:
    for tp in tps:
        if actual_number_of_trades == 0:
            ending_val,win_rate,wins,losses = 0,0,0,0
            ending_val4,win_rate4,wins4,losses4 = 0,0,0,0
        else:
            ending_val,win_rate,wins,losses = strategy_tester_buy(df,col = trade_strategy,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False,spread = spread)
            ending_val4,win_rate4,wins4,losses4 = strategy_tester_sell(df,col = trade_strategy,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False,spread = spread)   



        max_ending_val = max(ending_val,
               ending_val4)
        lst = [instrument,
               ending_val,
               ending_val4,
               max_ending_val,
                sl,
                tp,
                round(tp / sl,4),
               round(win_rate,4),
               round(win_rate4,4),
               actual_number_of_trades
              ]
        temp_lst.append(lst)

final_df = pd.DataFrame(temp_lst,columns = cols)
final_df = final_df.sort_values(by = 'max_ending_val',ascending = False)

e = time.time()
best_buy_support = final_df['buy_support_val'].max()
best_sell_support = final_df['sell_support_val'].max()
max_val_all = final_df['max_ending_val'].max()
print('MAX VAL:',round(max_val_all),'||best buy support:',round(best_buy_support),'||best sell support:',round(best_sell_support))
final_df.head()

NEW SHAPE 80
MAX VAL: 17170325 ||best buy support: 36200 ||best sell support: 17170325


,instrument,buy_support_val,sell_support_val,max_ending_val,sl,tp,RR,win_rate1,win_rate4,support_number_of_trades
10,EUR_USD,14743.0,17170325.0,17170325.0,0.00050,0.013,26.0000,0.0500,0.1375,80
11,EUR_USD,20953.0,8696100.5,8696100.5,0.00050,0.015,30.0000,0.0500,0.1125,80
22,EUR_USD,14461.0,3909881.0,3909881.0,0.00075,0.013,17.3333,0.0750,0.1625,80
23,EUR_USD,23947.0,3171216.0,3171216.0,0.00075,0.015,20.0000,0.0750,0.1375,80
9,EUR_USD,16432.5,3029468.0,3029468.0,0.00050,0.011,22.0000,0.0625,0.1375,80


In [95]:
final_df.head(20)

,instrument,buy_support_val,sell_support_val,max_ending_val,sl,tp,RR,win_rate1,win_rate4,support_number_of_trades
10,EUR_USD,14743.0,17170325.0,17170325.0,0.00050,0.013,26.0000,0.0500,0.1375,80
11,EUR_USD,20953.0,8696100.5,8696100.5,0.00050,0.015,30.0000,0.0500,0.1125,80
22,EUR_USD,14461.0,3909881.0,3909881.0,0.00075,0.013,17.3333,0.0750,0.1625,80
23,EUR_USD,23947.0,3171216.0,3171216.0,0.00075,0.015,20.0000,0.0750,0.1375,80
9,EUR_USD,16432.5,3029468.0,3029468.0,0.00050,0.011,22.0000,0.0625,0.1375,80
21,EUR_USD,14342.0,2521068.0,2521068.0,0.00075,0.011,14.6667,0.0875,0.1750,80
34,EUR_USD,25560.0,1560911.0,1560911.0,0.00100,0.013,13.0000,0.1000,0.1875,80
47,EUR_USD,18750.5,1467765.5,1467765.5,0.00125,0.015,12.0000,0.1000,0.1875,80
35,EUR_USD,23365.0,1427614.0,1427614.0,0.00100,0.015,15.0000,0.0875,0.1625,80
46,EUR_USD,15490.0,991370.0,991370.0,0.00125,0.013,10.4000,0.1125,0.2125,80


In [96]:
pair = 'AUD_USD'
df = load_df(pair = pair,granularity ='M5',start = datetime(2016,1,1,0,0,0),end = datetime(2022,7,31,0,0,0))
print(df.shape)
df = get_support(df,
                lookup_range = 300,
                stop_range = 10000,
                bound_pips = .001,
                bound_pips2 = .005
               )
print(df.shape)
df = support_indicator_2(df,candle_size = .0008)
trade_strategy = 'support_indicator2'


#trade_strategy = 'resistance_indicatorc'
num_trades = 400
risk_size = .01

starting_val = 10000
instrument = pair
timeframe = 'M5'
temp_lst = []
actual_number_of_trades = df[df[trade_strategy] == 1].shape[0]
sls = [.0005,.00075,.001,.00125,.0015,.002,.0025]
tps = [.001,.0015,.002,.0025,.003,.004,.005,.007,.009,.011,.013,.015]
spread = .0001   
sl = .0005
tp = .004
ending_val,win_rate,wins,losses = strategy_tester_buy(df,col = trade_strategy,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False,spread = spread)
ending_val4,win_rate4,wins4,losses4 = strategy_tester_sell(df,col = trade_strategy,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False,spread = spread) 

print(ending_val,ending_val4)

PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/AUD_USD_M5/2016-01-01_2022-07-31.csv
(486874, 25)
(486874, 25)
GETTING SUPPORT INDICATOR
STOP RANGE: 10000
TOTAL FUNCTION TIME: 1.839998233318329  MINUTES
SHAPE (198, 27)
(486874, 27)
NEW SHAPE 37
895.0 19222.0


In [105]:
def support_indicator_2(df,candle_size = .001):
    df['support_indicator2'] = 0
    new_col = df.columns.get_loc('support_indicator2')
    col = df.columns.get_loc('support_indicator')
    lookup_col = df.columns.get_loc('support_lookup')
    c_col = df.columns.get_loc('c')
    o_col = df.columns.get_loc('o')
    l_col = df.columns.get_loc('l')
    h_col = df.columns.get_loc('h')
    arr = df.values
    
    for i in range(arr.shape[0]):
        lookup_check = 0
        if arr[i,col] == 1:
            for j in range(i - 1,i + 40):
                if arr[j,c_col] < arr[i,c_col] and arr[j,o_col] - arr[j,c_col] >= candle_size and lookup_check == 0:
                    arr[j,new_col] = 1
                    arr[j,lookup_col] = arr[i,lookup_col]
                  #  arr[j,c_col] = arr[j,h_col]
                    lookup_check = 1
    df = pd.DataFrame(arr,columns = df.columns)
    print('NEW SHAPE',df[df['support_indicator2'] == 1].shape[0])
    
    return df

pair = 'EUR_USD'
df = load_df(pair = pair,granularity ='M5',start = datetime(2016,1,1,0,0,0),end = datetime(2022,7,31,0,0,0))
print(df.shape)
df = get_support(df,
                lookup_range = 300,
                stop_range = 10000,
                bound_pips = .001,
                bound_pips2 = .005
               )
print(df.shape)
df = support_indicator_2(df,candle_size = .001)
trade_strategy = 'support_indicator2'



cols = ['instrument',
           'buy_support_val',
           'sell_support_val',
            'max_ending_val',
            'sl',
            'tp',
            'RR',
           'win_rate1',
           'win_rate4',
            'support_number_of_trades'
          ]

trade_strategy = 'support_indicator2'


#trade_strategy = 'resistance_indicatorc'
num_trades = 400
risk_size = .01

starting_val = 10000
instrument = pair
timeframe = 'M5'
temp_lst = []
actual_number_of_trades = df[df[trade_strategy] == 1].shape[0]
sls = [.0005,.00075,.001,.00125,.0015,.002,.0025]
tps = [.001,.0015,.002,.0025,.003,.004,.005,.007,.009,.011,.013,.015]
spread = .0001      

for sl in sls:
    for tp in tps:
        if actual_number_of_trades == 0:
            ending_val,win_rate,wins,losses = 0,0,0,0
            ending_val4,win_rate4,wins4,losses4 = 0,0,0,0
        else:
            ending_val,win_rate,wins,losses = strategy_tester_buy(df,col = trade_strategy,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False,spread = spread)
            ending_val4,win_rate4,wins4,losses4 = strategy_tester_sell(df,col = trade_strategy,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False,spread = spread)   



        max_ending_val = max(ending_val,
               ending_val4)
        lst = [instrument,
               ending_val,
               ending_val4,
               max_ending_val,
                sl,
                tp,
                round(tp / sl,4),
               round(win_rate,4),
               round(win_rate4,4),
               actual_number_of_trades
              ]
        temp_lst.append(lst)

final_df = pd.DataFrame(temp_lst,columns = cols)
final_df = final_df.sort_values(by = 'max_ending_val',ascending = False)

e = time.time()
best_buy_support = final_df['buy_support_val'].max()
best_sell_support = final_df['sell_support_val'].max()
max_val_all = final_df['max_ending_val'].max()
print('MAX VAL:',round(max_val_all),'||best buy support:',round(best_buy_support),'||best sell support:',round(best_sell_support))
final_df.head()


PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/EUR_USD_M5/2016-01-01_2022-07-31.csv
(490023, 25)
(490023, 25)
GETTING SUPPORT INDICATOR
STOP RANGE: 10000
TOTAL FUNCTION TIME: 1.9161599318186442  MINUTES
SHAPE (213, 27)
(490023, 27)
NEW SHAPE 50
MAX VAL: 2322670 ||best buy support: 49073 ||best sell support: 2322670


,instrument,buy_support_val,sell_support_val,max_ending_val,sl,tp,RR,win_rate1,win_rate4,support_number_of_trades
47,EUR_USD,2513.0,2322670.0,2322670.0,0.00125,0.015,12.0000,0.06,0.20,50
59,EUR_USD,9451.0,1479890.0,1479890.0,0.00150,0.015,10.0000,0.10,0.22,50
46,EUR_USD,4837.5,485307.0,485307.0,0.00125,0.013,10.4000,0.08,0.20,50
58,EUR_USD,13723.5,476060.0,476060.0,0.00150,0.013,8.6667,0.12,0.22,50
45,EUR_USD,11577.0,272084.5,272084.5,0.00125,0.011,8.8000,0.12,0.20,50


In [101]:
final_df = final_df.sort_values(by = 'sell_support_val',ascending = False)
final_df.head(20)

,instrument,buy_support_val,sell_support_val,max_ending_val,sl,tp,RR,win_rate1,win_rate4,support_number_of_trades
0,EUR_USD,180.0,53575.0,53575.0,0.00050,0.0010,2.0000,0.0000,0.5714,21
24,EUR_USD,258.0,30403.0,30403.0,0.00100,0.0010,1.0000,0.0476,0.7143,21
12,EUR_USD,180.0,27523.0,27523.0,0.00075,0.0010,1.3333,0.0000,0.6190,21
36,EUR_USD,1653.0,21447.0,21447.0,0.00125,0.0010,0.8000,0.3333,0.7143,21
3,EUR_USD,180.0,20338.0,20338.0,0.00050,0.0025,5.0000,0.0000,0.2381,21
4,EUR_USD,180.0,18023.0,18023.0,0.00050,0.0030,6.0000,0.0000,0.1905,21
2,EUR_USD,180.0,17803.0,17803.0,0.00050,0.0020,4.0000,0.0000,0.2857,21
48,EUR_USD,4590.0,15571.0,15571.0,0.00150,0.0010,0.6667,0.5238,0.7143,21
26,EUR_USD,305.0,10873.5,10873.5,0.00100,0.0020,2.0000,0.0476,0.3810,21
60,EUR_USD,4809.0,10651.0,10651.0,0.00200,0.0010,0.5000,0.5714,0.7143,21


In [107]:
def support_indicator_2(df,candle_size = .001):
    df['support_indicator2'] = 0
    new_col = df.columns.get_loc('support_indicator2')
    col = df.columns.get_loc('support_indicator')
    lookup_col = df.columns.get_loc('support_lookup')
    c_col = df.columns.get_loc('c')
    o_col = df.columns.get_loc('o')
    l_col = df.columns.get_loc('l')
    h_col = df.columns.get_loc('h')
    arr = df.values
    
    for i in range(arr.shape[0]):
        lookup_check = 0
        if arr[i,col] == 1:
            for j in range(i - 1,i + 40):
                if arr[j,c_col] > arr[i,c_col] and arr[j,c_col] - arr[j,o_col] >= candle_size and lookup_check == 0:
                    arr[j,new_col] = 1
                    arr[j,lookup_col] = arr[i,lookup_col]
                  #  arr[j,c_col] = arr[j,h_col]
                    lookup_check = 1
    df = pd.DataFrame(arr,columns = df.columns)
    print('NEW SHAPE',df[df['support_indicator2'] == 1].shape[0])
    
    return df

all_pairs=['EUR_USD','GBP_JPY','USD_JPY','EUR_JPY','AUD_JPY','CAD_JPY','CHF_JPY',
                   'GBP_USD','USD_CAD','USD_CHF','AUD_USD',
                   'NZD_USD','AUD_CAD','AUD_CHF','AUD_NZD','CAD_CHF',
                   'EUR_AUD','EUR_CAD','EUR_CHF','NZD_CAD','NZD_CHF'] 
for pair in all_pairs:

    df = load_df(pair = pair,granularity ='M5',start = datetime(2016,1,1,0,0,0),end = datetime(2022,7,31,0,0,0))
    print(df.shape)
    if 'JPY' in pair:
        bound_pips = .001 * 100
        bound_pips2 = .005 * 100
        candle_size = .001 * 100

        spread = .0001 * 100 
        sl = 0.001 * 100 
        tp = 0.01 * 100 

    else:
        bound_pips = .001 
        bound_pips2 = .005
        candle_size = .001

        spread = .0001   
        sl = 0.001
        tp = 0.01     
        
    df = get_support(df,
                    lookup_range = 300,
                    stop_range = 10000,
                    bound_pips = bound_pips,
                    bound_pips2 = bound_pips2
                   )
    print(df.shape)
    df = support_indicator_2(df,candle_size = candle_size)
    trade_strategy = 'support_indicator2'


    #trade_strategy = 'resistance_indicatorc'
    num_trades = 400
    risk_size = .01

    starting_val = 10000
    instrument = pair
    timeframe = 'M5'
    temp_lst = []
    actual_number_of_trades = df[df[trade_strategy] == 1].shape[0]

    ending_val,win_rate,wins,losses = strategy_tester_buy(df,col = trade_strategy,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False,spread = spread)
    ending_val4,win_rate4,wins4,losses4 = strategy_tester_sell(df,col = trade_strategy,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False,spread = spread) 

    print(ending_val,ending_val4)   
    
    

PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/EUR_USD_M5/2016-01-01_2022-07-31.csv
(490023, 25)
(490023, 25)
GETTING SUPPORT INDICATOR
STOP RANGE: 10000
TOTAL FUNCTION TIME: 1.930351185798645  MINUTES
SHAPE (213, 27)
(490023, 27)
NEW SHAPE 53
6953.0 725.0
PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/GBP_JPY_M5/2016-01-01_2022-07-31.csv
(491385, 25)
(491385, 25)
GETTING SUPPORT INDICATOR
STOP RANGE: 10000
TOTAL FUNCTION TIME: 3.2261364698410033  MINUTES
SHAPE (454, 27)
(491385, 27)
NEW SHAPE 252
3724.0 6315.0
PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/USD_JPY_M5/2016-01-01_2022-07-31.csv
(490422, 25)
(490422, 25)
GETTING SUPPORT INDICATOR
STOP RANGE: 10000
TOTAL FUNCTION TIME: 1.9906525015830994  MINUTES
SHAPE (234, 27)
(490422, 27)
NEW SHAPE 70
4731.0 1491.0
PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/EUR_JPY_M5/2016-01-01_2022-07-31.csv
(490530, 25)
(490530, 25)
GETTING SUPPORT INDICATOR
STOP RAN

KeyboardInterrupt: 

In [108]:
def support_indicator_2(df,candle_size = .001):
    df['support_indicator2'] = 0
    new_col = df.columns.get_loc('support_indicator2')
    col = df.columns.get_loc('support_indicator')
    lookup_col = df.columns.get_loc('support_lookup')
    c_col = df.columns.get_loc('c')
    o_col = df.columns.get_loc('o')
    l_col = df.columns.get_loc('l')
    h_col = df.columns.get_loc('h')
    arr = df.values
    
    for i in range(arr.shape[0]):
        lookup_check = 0
        if arr[i,col] == 1:
            for j in range(i - 1,i + 40):
                if arr[j,c_col] > arr[i,c_col] and arr[j,c_col] - arr[j,l_col] >= candle_size and arr[j,o_col] >= arr[j,c_col] and lookup_check == 0:
                    arr[j,new_col] = 1
                    arr[j,lookup_col] = arr[i,lookup_col]
                  #  arr[j,c_col] = arr[j,h_col]
                    lookup_check = 1
    df = pd.DataFrame(arr,columns = df.columns)
    print('NEW SHAPE',df[df['support_indicator2'] == 1].shape[0])
    
    return df

all_pairs=['EUR_USD','GBP_JPY','USD_JPY','EUR_JPY','AUD_JPY','CAD_JPY','CHF_JPY',
                   'GBP_USD','USD_CAD','USD_CHF','AUD_USD',
                   'NZD_USD','AUD_CAD','AUD_CHF','AUD_NZD','CAD_CHF',
                   'EUR_AUD','EUR_CAD','EUR_CHF','NZD_CAD','NZD_CHF'] 
for pair in all_pairs:

    df = load_df(pair = pair,granularity ='M5',start = datetime(2016,1,1,0,0,0),end = datetime(2022,7,31,0,0,0))
    print(df.shape)
    if 'JPY' in pair:
        bound_pips = .001 * 100
        bound_pips2 = .005 * 100
        candle_size = .0007 * 100

        spread = .0001 * 100 
        sl = 0.001 * 100 
        tp = 0.01 * 100 

    else:
        bound_pips = .001 
        bound_pips2 = .005
        candle_size = .0007

        spread = .0001   
        sl = 0.001
        tp = 0.01     
        
    df = get_support(df,
                    lookup_range = 300,
                    stop_range = 10000,
                    bound_pips = bound_pips,
                    bound_pips2 = bound_pips2
                   )
    print(df.shape)
    df = support_indicator_2(df,candle_size = candle_size)
    trade_strategy = 'support_indicator2'


    #trade_strategy = 'resistance_indicatorc'
    num_trades = 400
    risk_size = .01

    starting_val = 10000
    instrument = pair
    timeframe = 'M5'
    temp_lst = []
    actual_number_of_trades = df[df[trade_strategy] == 1].shape[0]

    ending_val,win_rate,wins,losses = strategy_tester_buy(df,col = trade_strategy,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False,spread = spread)
    ending_val4,win_rate4,wins4,losses4 = strategy_tester_sell(df,col = trade_strategy,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False,spread = spread) 

    print(ending_val,ending_val4)   
    
    

PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/EUR_USD_M5/2016-01-01_2022-07-31.csv
(490023, 25)
(490023, 25)
GETTING SUPPORT INDICATOR
STOP RANGE: 10000
TOTAL FUNCTION TIME: 1.9233691175778707  MINUTES
SHAPE (213, 27)
(490023, 27)
NEW SHAPE 15
2412.0 342924.0
PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/GBP_JPY_M5/2016-01-01_2022-07-31.csv
(491385, 25)
(491385, 25)
GETTING SUPPORT INDICATOR
STOP RANGE: 10000
TOTAL FUNCTION TIME: 3.2524604320526125  MINUTES
SHAPE (454, 27)
(491385, 27)
NEW SHAPE 134
1642.0 15083.0
PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/USD_JPY_M5/2016-01-01_2022-07-31.csv
(490422, 25)
(490422, 25)
GETTING SUPPORT INDICATOR
STOP RANGE: 10000
TOTAL FUNCTION TIME: 2.002469197909037  MINUTES
SHAPE (234, 27)
(490422, 27)
NEW SHAPE 21
41285.0 47640.0
PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/EUR_JPY_M5/2016-01-01_2022-07-31.csv
(490530, 25)
(490530, 25)
GETTING SUPPORT INDICATOR
ST

KeyboardInterrupt: 

In [99]:
def support_indicator_2(df,candle_size = .001):
    df['support_indicator2'] = 0
    new_col = df.columns.get_loc('support_indicator2')
    col = df.columns.get_loc('support_indicator')
    lookup_col = df.columns.get_loc('support_lookup')
    c_col = df.columns.get_loc('c')
    o_col = df.columns.get_loc('o')
    l_col = df.columns.get_loc('l')
    h_col = df.columns.get_loc('h')
    arr = df.values
    
    for i in range(arr.shape[0]):
        lookup_check = 0
        if arr[i,col] == 1:
            for j in range(i - 1,i + 40):
                if arr[j,c_col] > arr[i,c_col] and lookup_check == 0 and arr[j,h_col] - arr[j,c_col] >= candle_size:
                    arr[j,new_col] = 1
                    arr[j,lookup_col] = arr[i,lookup_col]
                    lookup_check = 1
    df = pd.DataFrame(arr,columns = df.columns)
    print('NEW SHAPE',df[df['support_indicator2'] == 1].shape[0])
    
    return df

all_pairs=['EUR_USD','GBP_JPY','USD_JPY','EUR_JPY','AUD_JPY','CAD_JPY','CHF_JPY',
                   'GBP_USD','USD_CAD','USD_CHF','AUD_USD',
                   'NZD_USD','AUD_CAD','AUD_CHF','AUD_NZD','CAD_CHF',
                   'EUR_AUD','EUR_CAD','EUR_CHF','NZD_CAD','NZD_CHF'] 
for pair in all_pairs:

    df = load_df(pair = pair,granularity ='M5',start = datetime(2016,1,1,0,0,0),end = datetime(2022,7,31,0,0,0))
    print(df.shape)
    if 'JPY' in pair:
        bound_pips = .001 * 100
        bound_pips2 = .005 * 100
        candle_size = .0008 * 100

        spread = .0001 * 100 
        sl = .0005 * 100 
        tp = .004 * 100 
    else:
        bound_pips = .001 
        bound_pips2 = .005
        candle_size = .0008

        spread = .0001   
        sl = .0005
        tp = .004        
        
    df = get_support(df,
                    lookup_range = 300,
                    stop_range = 10000,
                    bound_pips = bound_pips,
                    bound_pips2 = bound_pips2
                   )
    print(df.shape)
    df = support_indicator_2(df,candle_size = candle_size)
    trade_strategy = 'support_indicator2'


    #trade_strategy = 'resistance_indicatorc'
    num_trades = 400
    risk_size = .01

    starting_val = 10000
    instrument = pair
    timeframe = 'M5'
    temp_lst = []
    actual_number_of_trades = df[df[trade_strategy] == 1].shape[0]

    ending_val,win_rate,wins,losses = strategy_tester_buy(df,col = trade_strategy,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False,spread = spread)
    ending_val4,win_rate4,wins4,losses4 = strategy_tester_sell(df,col = trade_strategy,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False,spread = spread) 

    print(ending_val,ending_val4)   
    
    
    

PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/EUR_USD_M5/2016-01-01_2022-07-31.csv
(490023, 25)
(490023, 25)
GETTING SUPPORT INDICATOR
STOP RANGE: 10000
TOTAL FUNCTION TIME: 1.9462398489316304  MINUTES
SHAPE (213, 27)
(490023, 27)
NEW SHAPE 80
1439.5 412863.0
PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/GBP_JPY_M5/2016-01-01_2022-07-31.csv
(491385, 25)
(491385, 25)
GETTING SUPPORT INDICATOR
STOP RANGE: 10000
TOTAL FUNCTION TIME: 3.236211395263672  MINUTES
SHAPE (454, 27)
(491385, 27)
NEW SHAPE 311
1114.0 59657.5
PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/USD_JPY_M5/2016-01-01_2022-07-31.csv
(490422, 25)
(490422, 25)
GETTING SUPPORT INDICATOR
STOP RANGE: 10000
TOTAL FUNCTION TIME: 1.9969086647033691  MINUTES
SHAPE (234, 27)
(490422, 27)
NEW SHAPE 86
499.0 82811.0
PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/EUR_JPY_M5/2016-01-01_2022-07-31.csv
(490530, 25)
(490530, 25)
GETTING SUPPORT INDICATOR
STOP

KeyboardInterrupt: 

In [83]:
df.head()

,time,volume,o,h,l,c,utc_timestamp,est_timestamp,smma_21,smma_50,...,new_york,london,delta_vector,delta_max,upper_wick,lower_wick,delta,support_lookup,support_indicator,support_indicator2
0,2016-01-03 22:00:00,6,1.08743,1.08746,1.08724,1.08726,2016-01-03 22:00:00+00:00,2016-01-03 17:00:00-05:00,1.08726,1.08726,...,0,0,-0.00017,0.00022,0.00003,0.00002,0.00017,0,0,0
1,2016-01-03 22:05:00,53,1.08735,1.08738,1.0871,1.0871,2016-01-03 22:05:00+00:00,2016-01-03 17:05:00-05:00,1.0871,1.0871,...,0,0,-0.00025,0.00028,0.00003,0.0,0.00025,0,0,0
2,2016-01-03 22:10:00,21,1.08715,1.08715,1.08674,1.08681,2016-01-03 22:10:00+00:00,2016-01-03 17:10:00-05:00,1.08681,1.08681,...,0,0,-0.00034,0.00041,0.0,0.00007,0.00034,0,0,0
3,2016-01-03 22:15:00,19,1.08671,1.08694,1.08669,1.0868,2016-01-03 22:15:00+00:00,2016-01-03 17:15:00-05:00,1.0868,1.0868,...,0,0,0.00009,0.00025,0.00014,0.00002,0.00009,0,0,0
4,2016-01-03 22:20:00,47,1.08684,1.08717,1.0867,1.0867,2016-01-03 22:20:00+00:00,2016-01-03 17:20:00-05:00,1.0867,1.0867,...,0,0,-0.00014,0.00047,0.00033,0.0,0.00014,0,0,0


In [85]:
def viz_support(df,col,start,end,ema):
    print('SELL INTO SUPPORT!!!!!')
    def print_support_example(df,col,ind_list,ind = 30,delta = 240,ema = True):
        i = ind_list[ind]   
        x = df['support_lookup'].iloc[i]


        print('SUPPORT ROW:',x,' SUPPORT NEXT TOUCH:',i, ' SUPPORT CLOSE VAL:',df['c'].iloc[x])

        fig = go.Figure(data=go.Candlestick(x=df.iloc[x - delta:i + delta,:]['time'],
                            open=df.iloc[x - delta:i + delta,:]['o'],
                            high=df.iloc[x - delta:i + delta,:]['h'],
                            low=df.iloc[x - delta:i + delta,:]['l'],
                            close=df.iloc[x - delta:i + delta,:]['c']))   


        fig.add_vrect(x0=df['time'].iloc[i - 1], x1=df['time'].iloc[i+1], 
                      annotation_text="trigger point" , annotation_position="top left",
                      fillcolor="green", opacity=0.25, line_width=0)

        fig.add_shape(type='line',
                        x0=df.iloc[x,:]['time'],
                        y0=df.iloc[x,:]['c'],
                        x1=df.iloc[i,:]['time'],
                        y1=df.iloc[x ,:]['c'],
                        line=dict(color='Red',),
                        xref='x',
                        yref='y'
        )
        if ema:

            fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['ema_42'],line=dict(color='Yellow',)))
            fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['ema_150'],line=dict(color='Blue',)))
            fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['ema_600'],line=dict(color='Black',)))
        else:
            fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['smma_21'],line=dict(color='Yellow',)))
            fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['smma_50'],line=dict(color='Blue',)))
            fig.add_trace(go.Line(x=df.iloc[x - delta:i + delta,:]['time'],y=df.iloc[x - delta:i + delta,:]['smma_200'],line=dict(color='Black',)))
        try:
            plot(fig)
        except:
            from plotly.offline import plot
            plot(fig)
            
            
    temp_df = df[df[col] == 1]
    print('SHAPE:',temp_df.shape)
    ind_list = temp_df.index
    for i in range(start,end):
        print(i)

        #i = 3
        print_support_example(df,col,ind_list,ind = i,delta = 1000,ema = ema)
        i += 1

viz_support(df = df,col = 'support_indicator2',start = 10,end = 20,ema = True)

SELL INTO SUPPORT!!!!!
SHAPE: (49, 28)
10
SUPPORT ROW: 66034  SUPPORT NEXT TOUCH: 66880  SUPPORT CLOSE VAL: 1.05738


/opt/anaconda3/lib/python3.8/site-packages/plotly/graph_objs/_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




11
SUPPORT ROW: 68337  SUPPORT NEXT TOUCH: 70336  SUPPORT CLOSE VAL: 1.05615
12
SUPPORT ROW: 79142  SUPPORT NEXT TOUCH: 79495  SUPPORT CLOSE VAL: 1.07134
13
SUPPORT ROW: 80714  SUPPORT NEXT TOUCH: 81507  SUPPORT CLOSE VAL: 1.07391
14
SUPPORT ROW: 120624  SUPPORT NEXT TOUCH: 121183  SUPPORT CLOSE VAL: 1.16918
15
SUPPORT ROW: 133821  SUPPORT NEXT TOUCH: 135604  SUPPORT CLOSE VAL: 1.17313
16
SUPPORT ROW: 151698  SUPPORT NEXT TOUCH: 152079  SUPPORT CLOSE VAL: 1.21995
17
SUPPORT ROW: 182098  SUPPORT NEXT TOUCH: 182463  SUPPORT CLOSE VAL: 1.17336
18
SUPPORT ROW: 188457  SUPPORT NEXT TOUCH: 192520  SUPPORT CLOSE VAL: 1.16165
19
SUPPORT ROW: 203104  SUPPORT NEXT TOUCH: 203971  SUPPORT CLOSE VAL: 1.17262


In [109]:
df.columns

Index(['time', 'volume', 'o', 'h', 'l', 'c', 'utc_timestamp', 'est_timestamp',
       'smma_21', 'smma_50', 'smma_200', 'ema_42', 'ema_150', 'ema_600',
       'bearish_engulfing', 'bullish_engulfing', 'bearish_tls', 'bullish_tls',
       'new_york', 'london', 'delta_vector', 'delta_max', 'upper_wick',
       'lower_wick', 'delta', 'support_lookup', 'support_indicator'],
      dtype='object')

In [125]:
def get_support(df,
                lookup_range = 1500,
                stop_range = 2200,
                bound_pips = .001,
                bound_pips2 = .005,
                ema = True
               ):
    print('GETTING SUPPORT INDICATOR')
    s = time.time()
    c_col = df.columns.get_loc('c')
    o_col = df.columns.get_loc('o')
    l_col = df.columns.get_loc('l')
    h_col = df.columns.get_loc('h')
    d_col = df.columns.get_loc('delta')
    if ema:
        ma1 = df.columns.get_loc('ema_42')
        ma2 = df.columns.get_loc('ema_150')
        ma3 = df.columns.get_loc('ema_600')
    else:
        ma1 = df.columns.get_loc('smma_21')
        ma2 = df.columns.get_loc('smma_50')
        ma3 = df.columns.get_loc('smma_200')
        
    df['support_lookup'] = 0
    df['support_indicator'] = 0

    lookup_col = df.columns.get_loc('support_lookup')
    new_col = df.columns.get_loc('support_indicator')

    reversal_col = df.columns.get_loc('c')   

    arr = df.values
    min_count = 0

    print('STOP RANGE:',stop_range)
    c = 0 

    for i in range(lookup_range,arr.shape[0]):
       # try:
        #DEFINE SUPPORT/RESISTANCE as the MIN or MAX of a lookup range
        #min_ = min(arr[i - lookup_range : i + lookup_range2,reversal_col])
        min_ = min(arr[i - lookup_range : i + 1,reversal_col])

        
        reversal_ind = 0
        if arr[i,reversal_col] == min_:
            for k in range(i,i + stop_range):
                if k >= arr.shape[0]:
                    ind = 0
                    break
                if arr[k,reversal_col] < min_:
                    break
                if arr[k,reversal_col] - min_ >= bound_pips2 :
                    reversal_ind = 1
                    ind = k
                    break
                
        # If the current val is at that reversal point:
        # Check and make sure we have an adequate movement off of that reversal point
        # this is defined by the pip_range and pip threshold, pip threshold is a user defined parameter
        if reversal_ind == 1 and ind != 0:
            lookup_ind = i
            val = arr[i,reversal_col]
            lookup_check = 0
            # Now that all of these conditions are met
            # We need to look into the future to see if we hit this reversal level, this is defined by the stop range
            if ind + stop_range < df.shape[0]:
                for j in range(ind,ind + stop_range):

                    lower_bound = val - bound_pips 
                    upper_bound = val + (bound_pips/4)                    
                    cur_val = arr[j, reversal_col]
                    if arr[j,reversal_col] < lower_bound - bound_pips:
                        lookup_check = 1
                        break                    
                    
                    if lookup_check == 0 and \
                    cur_val <= upper_bound and \
                    cur_val >= lower_bound:
                        
                        if j + (stop_range * 3) >= df.shape[0]:
                            loop_val = df.shape[0]
                        else:
                            loop_val = j + (stop_range * 3)
                        for k in range(j, loop_val):
                            cur_val2 = arr[k, reversal_col]
                            if arr[k,reversal_col] < lower_bound - bound_pips:
                                lookup_check = 1
                                break
                            
                            if lookup_check == 0 and \
                            cur_val2 > val and \
                            arr[k,ma1] > arr[k,ma2] and arr[k,ma2] > arr[k,ma3] and arr[k,c_col] <= arr[k,ma3]: #arr[k,o_col] - arr[k,c_col] >= .0015:                     
                                arr[k,new_col] = 1
                                arr[k,lookup_col] = lookup_ind
                                lookup_check = 1
                                break                    
                    
                    

    e = time.time()
    print('TOTAL FUNCTION TIME:',(e-s)/60,' MINUTES')
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['support_indicator'] == 1].shape)
    return df



pair = 'EUR_USD'
df = load_df(pair = pair,granularity ='M5',start = datetime(2016,1,1,0,0,0),end = datetime(2022,7,31,0,0,0))
print(df.shape)
df = get_support(df,
                lookup_range = 300,
                stop_range = 10000,
                bound_pips = .0015,
                bound_pips2 = .007,
                 ema = True
               )
print(df.shape)
df.head()


PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/EUR_USD_M5/2016-01-01_2022-07-31.csv
(490023, 25)
(490023, 25)
GETTING SUPPORT INDICATOR
STOP RANGE: 10000
TOTAL FUNCTION TIME: 0.08006259997685751  MINUTES
SHAPE (96, 27)
(490023, 27)


,time,volume,o,h,l,c,utc_timestamp,est_timestamp,smma_21,smma_50,...,bullish_tls,new_york,london,delta_vector,delta_max,upper_wick,lower_wick,delta,support_lookup,support_indicator
0,2016-01-03 22:00:00,6,1.08743,1.08746,1.08724,1.08726,2016-01-03 22:00:00+00:00,2016-01-03 17:00:00-05:00,1.08726,1.08726,...,0,0,0,-0.00017,0.00022,0.00003,0.00002,0.00017,0,0
1,2016-01-03 22:05:00,53,1.08735,1.08738,1.0871,1.0871,2016-01-03 22:05:00+00:00,2016-01-03 17:05:00-05:00,1.0871,1.0871,...,0,0,0,-0.00025,0.00028,0.00003,0.0,0.00025,0,0
2,2016-01-03 22:10:00,21,1.08715,1.08715,1.08674,1.08681,2016-01-03 22:10:00+00:00,2016-01-03 17:10:00-05:00,1.08681,1.08681,...,0,0,0,-0.00034,0.00041,0.0,0.00007,0.00034,0,0
3,2016-01-03 22:15:00,19,1.08671,1.08694,1.08669,1.0868,2016-01-03 22:15:00+00:00,2016-01-03 17:15:00-05:00,1.0868,1.0868,...,0,0,0,0.00009,0.00025,0.00014,0.00002,0.00009,0,0
4,2016-01-03 22:20:00,47,1.08684,1.08717,1.0867,1.0867,2016-01-03 22:20:00+00:00,2016-01-03 17:20:00-05:00,1.0867,1.0867,...,0,0,0,-0.00014,0.00047,0.00033,0.0,0.00014,0,0


In [126]:

viz_support(df = df,col = 'support_indicator',start = 10,end = 20,ema = True)

SELL INTO SUPPORT!!!!!
SHAPE: (96, 27)
10
SUPPORT ROW: 55940  SUPPORT NEXT TOUCH: 56808  SUPPORT CLOSE VAL: 1.11556


/opt/anaconda3/lib/python3.8/site-packages/plotly/graph_objs/_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




11
SUPPORT ROW: 58484  SUPPORT NEXT TOUCH: 59378  SUPPORT CLOSE VAL: 1.09868
12
SUPPORT ROW: 67734  SUPPORT NEXT TOUCH: 68602  SUPPORT CLOSE VAL: 1.05684
13
SUPPORT ROW: 67089  SUPPORT NEXT TOUCH: 69712  SUPPORT CLOSE VAL: 1.05242
14
SUPPORT ROW: 71512  SUPPORT NEXT TOUCH: 73200  SUPPORT CLOSE VAL: 1.03732
15
SUPPORT ROW: 73806  SUPPORT NEXT TOUCH: 76928  SUPPORT CLOSE VAL: 1.03741
16
SUPPORT ROW: 80714  SUPPORT NEXT TOUCH: 81431  SUPPORT CLOSE VAL: 1.07391
17
SUPPORT ROW: 83529  SUPPORT NEXT TOUCH: 85175  SUPPORT CLOSE VAL: 1.05234
18
SUPPORT ROW: 84949  SUPPORT NEXT TOUCH: 87509  SUPPORT CLOSE VAL: 1.0496
19
SUPPORT ROW: 89092  SUPPORT NEXT TOUCH: 94808  SUPPORT CLOSE VAL: 1.0601200000000002


In [182]:
def get_support(df,
                lookup_range = 1500,
                stop_range = 2200,
                bound_pips = .001,
                bound_pips2 = .005,
                ema = True
               ):
    print('GETTING SUPPORT INDICATOR')
    s = time.time()
    c_col = df.columns.get_loc('c')
    o_col = df.columns.get_loc('o')
    l_col = df.columns.get_loc('l')
    h_col = df.columns.get_loc('h')
    d_col = df.columns.get_loc('delta')
    if ema:
        ma1 = df.columns.get_loc('ema_42')
        ma2 = df.columns.get_loc('ema_150')
        ma3 = df.columns.get_loc('ema_600')
    else:
        ma1 = df.columns.get_loc('smma_21')
        ma2 = df.columns.get_loc('smma_50')
        ma3 = df.columns.get_loc('smma_200')
        
    df['support_lookup'] = 0
    df['support_indicator'] = 0

    lookup_col = df.columns.get_loc('support_lookup')
    new_col = df.columns.get_loc('support_indicator')

    reversal_col = df.columns.get_loc('c')   

    arr = df.values
    min_count = 0

    print('STOP RANGE:',stop_range)
    c = 0 

    for i in range(lookup_range,arr.shape[0]):
       # try:
        #DEFINE SUPPORT/RESISTANCE as the MIN or MAX of a lookup range
        #min_ = min(arr[i - lookup_range : i + lookup_range2,reversal_col])
        min_ = min(arr[i - lookup_range : i + 1,reversal_col])
        if arr[i,o_col] > arr[i,c_col] and arr[i,c_col] - arr[i,l_col] >= bound_pips and arr[i + 1,l_col] <= arr[i,l_col]:
            arr[i,new_col] = 1
            arr[i,c_col] = arr[i,l_col]
        
          
            
                    
                    

    e = time.time()
    print('TOTAL FUNCTION TIME:',(e-s)/60,' MINUTES')
    df = pd.DataFrame(arr,columns = df.columns)
    print('SHAPE',df[df['support_indicator'] == 1].shape)
    return df



pair = 'EUR_USD'
df = load_df(pair = pair,granularity ='M5',start = datetime(2016,1,1,0,0,0),end = datetime(2022,7,31,0,0,0))
print(df.shape)
bound_pips_ = .001
bound_pips_2 = .005
df = get_support(df,
                lookup_range = 500,
                stop_range = 10000,
                bound_pips = bound_pips_,
                bound_pips2 = bound_pips_2,
                 ema = True
               )
print(df.shape)
df.head()

PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/EUR_USD_M5/2016-01-01_2022-07-31.csv
(490023, 25)
(490023, 25)
GETTING SUPPORT INDICATOR
STOP RANGE: 10000
TOTAL FUNCTION TIME: 0.09554248253504435  MINUTES
SHAPE (69, 27)
(490023, 27)


,time,volume,o,h,l,c,utc_timestamp,est_timestamp,smma_21,smma_50,...,bullish_tls,new_york,london,delta_vector,delta_max,upper_wick,lower_wick,delta,support_lookup,support_indicator
0,2016-01-03 22:00:00,6,1.08743,1.08746,1.08724,1.08726,2016-01-03 22:00:00+00:00,2016-01-03 17:00:00-05:00,1.08726,1.08726,...,0,0,0,-0.00017,0.00022,0.00003,0.00002,0.00017,0,0
1,2016-01-03 22:05:00,53,1.08735,1.08738,1.0871,1.0871,2016-01-03 22:05:00+00:00,2016-01-03 17:05:00-05:00,1.0871,1.0871,...,0,0,0,-0.00025,0.00028,0.00003,0.0,0.00025,0,0
2,2016-01-03 22:10:00,21,1.08715,1.08715,1.08674,1.08681,2016-01-03 22:10:00+00:00,2016-01-03 17:10:00-05:00,1.08681,1.08681,...,0,0,0,-0.00034,0.00041,0.0,0.00007,0.00034,0,0
3,2016-01-03 22:15:00,19,1.08671,1.08694,1.08669,1.0868,2016-01-03 22:15:00+00:00,2016-01-03 17:15:00-05:00,1.0868,1.0868,...,0,0,0,0.00009,0.00025,0.00014,0.00002,0.00009,0,0
4,2016-01-03 22:20:00,47,1.08684,1.08717,1.0867,1.0867,2016-01-03 22:20:00+00:00,2016-01-03 17:20:00-05:00,1.0867,1.0867,...,0,0,0,-0.00014,0.00047,0.00033,0.0,0.00014,0,0


In [183]:




cols = ['instrument',
           'buy_support_val',
           'sell_support_val',
            'max_ending_val',
            'sl',
            'tp',
            'RR',
           'win_rate1',
           'win_rate4',
            'support_number_of_trades'
          ]

trade_strategy = 'support_indicator'


#trade_strategy = 'resistance_indicatorc'
num_trades = 400
risk_size = .01

starting_val = 10000
instrument = pair
timeframe = 'M5'
temp_lst = []
actual_number_of_trades = df[df[trade_strategy] == 1].shape[0]
sls = [.0005,.00075,.001,.00125,.0015,.002,.0025]
tps = [.001,.0015,.002,.0025,.003,.004,.005,.007,.009,.011,.013,.015]
spread = .0001      

for sl in sls:
    for tp in tps:
        if actual_number_of_trades == 0:
            ending_val,win_rate,wins,losses = 0,0,0,0
            ending_val4,win_rate4,wins4,losses4 = 0,0,0,0
        else:
            ending_val,win_rate,wins,losses = strategy_tester_buy(df,col = trade_strategy,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False,spread = spread)
            ending_val4,win_rate4,wins4,losses4 = strategy_tester_sell(df,col = trade_strategy,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False,spread = spread)   



        max_ending_val = max(ending_val,
               ending_val4)
        lst = [instrument,
               ending_val,
               ending_val4,
               max_ending_val,
                sl,
                tp,
                round(tp / sl,4),
               round(win_rate,4),
               round(win_rate4,4),
               actual_number_of_trades
              ]
        temp_lst.append(lst)

final_df = pd.DataFrame(temp_lst,columns = cols)
final_df = final_df.sort_values(by = 'max_ending_val',ascending = False)

e = time.time()
best_buy_support = final_df['buy_support_val'].max()
best_sell_support = final_df['sell_support_val'].max()
max_val_all = final_df['max_ending_val'].max()
print('MAX VAL:',round(max_val_all),'||best buy support:',round(best_buy_support),'||best sell support:',round(best_sell_support))
final_df.head()



MAX VAL: 33380 ||best buy support: 33380 ||best sell support: 11278


,instrument,buy_support_val,sell_support_val,max_ending_val,sl,tp,RR,win_rate1,win_rate4,support_number_of_trades
36,EUR_USD,33380.0,319.5,33380.0,0.00125,0.001,0.8000,0.7826,0.0870,69
60,EUR_USD,26453.5,5877.0,26453.5,0.00200,0.001,0.5000,0.8696,0.6087,69
72,EUR_USD,24831.0,9265.0,24831.0,0.00250,0.001,0.4000,0.9130,0.7246,69
48,EUR_USD,24525.0,1375.0,24525.0,0.00150,0.001,0.6667,0.7826,0.3188,69
24,EUR_USD,23565.0,200.0,23565.0,0.00100,0.001,1.0000,0.6812,0.0145,69


In [ ]:
                bound_pips = .001,
                bound_pips2 = .0075,

In [184]:
all_pairs=['EUR_USD','GBP_JPY','USD_JPY','EUR_JPY','AUD_JPY','CAD_JPY','CHF_JPY',
                   'GBP_USD','USD_CAD','USD_CHF','AUD_USD',
                   'NZD_USD','AUD_CAD','AUD_CHF','AUD_NZD','CAD_CHF',
                   'EUR_AUD','EUR_CAD','EUR_CHF','NZD_CAD','NZD_CHF'] 
for pair in all_pairs:

    df = load_df(pair = pair,granularity ='M5',start = datetime(2016,1,1,0,0,0),end = datetime(2022,7,31,0,0,0))
    print(df.shape)
    if 'JPY' in pair:
        bound_pips = bound_pips_ * 100
        bound_pips2 = bound_pips_2 * 100

        spread = .0001 * 100 
        sl = .00125 * 100 
        tp = .01 * 100 
    else:
        bound_pips = bound_pips_
        bound_pips2 = bound_pips_2 

        spread = .0001   
        sl = .00125
        tp = .01
  
    df = get_support(df,
                    lookup_range = 500,
                    stop_range = 10000,
                    bound_pips = bound_pips,
                    bound_pips2 = bound_pips2
                   )
    print(df.shape)

    trade_strategy = 'support_indicator'


    #trade_strategy = 'resistance_indicatorc'
    num_trades = 400
    risk_size = .01

    starting_val = 10000
    instrument = pair
    timeframe = 'M5'
    temp_lst = []
    actual_number_of_trades = df[df[trade_strategy] == 1].shape[0]

    ending_val,win_rate,wins,losses = strategy_tester_buy(df,col = trade_strategy,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False,spread = spread)
    ending_val4,win_rate4,wins4,losses4 = strategy_tester_sell(df,col = trade_strategy,sl = sl,tp = tp,num_trades = num_trades,risk_size = risk_size,entry = 'c',pr = False,spread = spread) 

    print(ending_val,ending_val4)
    
    
    

PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/EUR_USD_M5/2016-01-01_2022-07-31.csv
(490023, 25)
(490023, 25)
GETTING SUPPORT INDICATOR
STOP RANGE: 10000
TOTAL FUNCTION TIME: 0.09524766604105632  MINUTES
SHAPE (69, 27)
(490023, 27)
1798.0 437.0
PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/GBP_JPY_M5/2016-01-01_2022-07-31.csv
(491385, 25)
(491385, 25)
GETTING SUPPORT INDICATOR
STOP RANGE: 10000
TOTAL FUNCTION TIME: 0.09415464798609416  MINUTES
SHAPE (775, 27)
(491385, 27)
5799.0 372.0
PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/USD_JPY_M5/2016-01-01_2022-07-31.csv
(490422, 25)
(490422, 25)
GETTING SUPPORT INDICATOR
STOP RANGE: 10000
TOTAL FUNCTION TIME: 0.09646678368250529  MINUTES
SHAPE (123, 27)
(490422, 27)
5574.0 180.0
PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/EUR_JPY_M5/2016-01-01_2022-07-31.csv
(490530, 25)
(490530, 25)
GETTING SUPPORT INDICATOR
STOP RANGE: 10000
TOTAL FUNCTION TIME: 0.096930

KeyboardInterrupt: 